In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Conv1D,Input,MaxPooling1D,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from collections import deque
import numpy as np
import  random
from tqdm import tqdm
import os
import time
import dataframe_image as dfi
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
#define the default font sizes:
plt.rc('font', size=12)
plt.rc('axes', labelsize=14, titlesize=12)
plt.rc('legend', fontsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
from pathlib import Path

# save the figures
IMAGES_PATH = Path() / "images" / "final"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=1200):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
weather=pd.read_csv("C:\\Users\\user\\Desktop\\iman\\Master\\RoboticsAndControlSystems\\Project\\Data\\Weather\\Weather.csv",parse_dates=["%time"],low_memory=False)

In [ ]:
for weather_attribute in weather.columns:
    x=weather.iloc[(np.where(weather[weather_attribute].isnull()))][weather_attribute]
    for i in reversed(x.index):
        if i+(12*24)< int(weather.index[-1]):
            #print(i+(12*24))
            weather.loc[i,weather_attribute]= weather.iloc[i+12*24][weather_attribute]
            #print(i+(12*24),weather.loc[i,weather_attribute])
for weather_attribute in weather.columns:  
    x=weather.iloc[(np.where(weather[weather_attribute].isnull()))][weather_attribute]
    for i in x.index:
        if i-(12*24)>= 0:
            #print(i+(12*24))
            weather.loc[i,weather_attribute]= weather.iloc[i-12*24][weather_attribute]
            #print(i+(12*24),weather.loc[i,weather_attribute])

print(weather.isnull().sum())

In [ ]:
weather=weather.set_index('%time').astype(float)

In [ ]:
weather.isnull().sum()

In [ ]:
weather.head()

In [ ]:
#def convert_to_preferred_format(sec):
#   sec = sec % (24 * 3600)
#   hour = sec // 3600
#   sec %= 3600
#   min = sec // 60
#   sec %= 60
#   print("seconds value in hours:",hour)
#   print("seconds value in minutes:",min)
#   return "%02d:%02d:%02d" % (hour, min, sec) 
#
#timestamp = weather['%time']
#print("Time in preferred format :-",convert_to_preferred_format(timestamp[0]))
#
#from datetime import datetime
# 
# 
#timestamp = weather['%time']
#dt_obj = datetime.fromtimestamp(timestamp[0])
# 
#print("date_time:",dt_obj)
#print("type of dt:",type(dt_obj))
#import time 
#time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.gmtime(timestamp[0]))
#import datetime; 
#datetime.datetime.utcfromtimestamp(timestamp[0]).replace(tzinfo=datetime.timezone.utc)

In [ ]:
#weather=weather.set_index('%time')

In [ ]:
weather.index

In [ ]:
weather.corr()

In [ ]:
weather.hist(bins=50,figsize=(20,15),column=weather.columns)

In [ ]:
weather.describe()

In [ ]:
labels = ['rain', 'no rain']
values=[(weather['Rain']==1).sum(),(weather['Rain']==0).sum()]
weather['Rain'][weather['Rain']<0.2]=0
weather['Rain'][weather['Rain']>0.2]=1

values=[(weather['Rain']==1).sum(),(weather['Rain']==0).sum()]

fig,ax=plt.subplots()
ax.bar(labels,values)
ax.set_ylabel('rain')
ax.set_title('Rain')
plt.show


In [ ]:
corr_matrix = weather.corr()['Rain'].sort_values(ascending=False)
corr_matrix


In [ ]:
#from pandas.plotting import scatter_matrix
#attributes = weather.columns.drop('Rain')
#scatter_matrix(weather[attributes], figsize=(15, 10))
#plt.show()

In [ ]:
#from matplotlib import pyplot as plt
#T=.1
##weather['Rain'].iloc[weather['Rain']<T]=0
##weather['Rain'].iloc[weather['Rain']>T]=1
#for col in attributes:
#    weather.boxplot(column=col, by='Rain', figsize=(6,6))
#    plt.title(col)
#plt.show()

In [ ]:
green_house_climate=pd.read_csv("C:\\Users\\user\\Desktop\\iman\\Master\\RoboticsAndControlSystems\\Project\\Data\\Automatoes\\GreenhouseClimate.csv",parse_dates=["%time"],low_memory=False).set_index('%time').astype(float)

In [ ]:
green_house_climate.head(5)
green_house_climate['Water_Quantity']=green_house_climate['Cum_irr']
green_house_climate['Duration_of_Irrigation']=green_house_climate['water_sup']
green_house_climate['Irrigation_Time_Intervals']=green_house_climate['water_sup_intervals_vip_min']

green_house_climate.drop(columns=['water_sup','Cum_irr','water_sup_intervals_vip_min'],inplace=True)

In [ ]:
green_house_climate.columns

In [ ]:
#green_house_climate.drop(columns=[  'AssimLight', 'BlackScr', 
#       'EnScr',  'PipeGrow', 'PipeLow', 
#       'Tot_PAR_Lamps', 'VentLee', 'Ventwind', 'assim_sp', 'assim_vip',
#        'co2_sp', 'co2_vip', 'dx_sp', 'dx_vip', 'int_blue_sp',
#       'int_blue_vip', 'int_farred_sp', 'int_farred_vip', 'int_red_sp',
#       'int_red_vip', 'int_white_sp', 'int_white_vip', 
#       'scr_blck_sp', 'scr_blck_vip', 'scr_enrg_sp', 'scr_enrg_vip',
#       't_grow_min_sp', 't_grow_min_vip', 't_heat_sp', 't_heat_vip',
#       't_rail_min_sp', 't_rail_min_vip', 't_vent_sp', 't_ventlee_vip',
#       't_ventwind_vip',  'window_pos_lee_sp','water_sup_intervals_sp_min',
#       'window_pos_lee_vip'],inplace=True)

In [ ]:
green_house_climate.info()
green_house_climate.head()

In [ ]:
#green_house_climate[np.where(green_house_climate.isnull()==True)]
#print(green_house_climate['Water_Quantity'].value_counts())

#print(green_house_climate['Irrigation_Time_Intervals'].value_counts())


In [ ]:
green_house_climate.hist(bins=10,figsize=(100,85),grid=[5,5],column=green_house_climate.columns)

In [ ]:
# Plot a histogram for each numeric feature
numeric_feature=green_house_climate.columns
for col in numeric_feature:
    fig = plt.figure(figsize=(10, 7))
    ax = fig.gca()
    feature = green_house_climate[col]
    feature.hist(bins=30, ax = ax)
    ax.axvline(feature.mean(), color='magenta', linestyle='dashed', linewidth=2)
    ax.axvline(feature.median(), color='cyan', linestyle='dashed', linewidth=2)
    ax.set_title(col)
plt.show()

In [ ]:
#Check Outliers in numarical features by using boxplot :
#from matplotlib import pyplot as plt
#
#for col in green_house_climate.columns:
#    green_house_climate.boxplot(column=col, figsize=(3,3))
#    plt.title(col)
#    plt.show()

In [ ]:
#weather.set_index('%time')
weather.info()

In [ ]:
#for num_attribute in green_house_climate.columns:
#        for weather_attribute in weather.columns:
#            plt.scatter(x=weather[weather_attribute],y=green_house_climate[num_attribute])
#            plt.show()

#find a trend 
#weather['time']=weather.index
##print(weather['%time'].astype('string').str.split('.',expand=True).astype('int')[0])
#x=weather['time'].astype('string').str.split('/',expand=True).astype('int')[1]
#weather['time']=x



#print('weather[time]',weather.index)
#print('************')
#weather_attributes=weather.columns.drop(['time'])
for weather_attribute in weather.columns:
            fig = plt.figure(figsize=(10, 7))
            ax = fig.gca()
            dayly_avg = weather[weather_attribute].resample('H').mean()  # compute the mean for each hour
            #print(dayly_avg)
            period=slice("2019-12-16 00:00:00", "2020-05-30 00:00:00")
            rolling_average_dayly = dayly_avg[period].rolling(window=24*10).mean()
            #dayly_avg=weather.groupby(by=['time'])[weather_attribute].mean()
            #print(weather_attribute,weather.groupby(by=['time'])[weather_attribute].mean())
            weather[weather_attribute].plot(ax=ax,ylabel=weather_attribute,xlabel='Day Index',grid='True', marker=".", figsize=(18, 6))
            
            #rolling_average_dayly = dayly_avg.rolling(window=10).mean()
            #print(weather_attribute,rolling_average_dayly)
            rolling_average_dayly.plot(grid=True)
            ax.set_xlabel(xlabel='%time')
            ax.set_ylabel(ylabel=weather_attribute)
            ax.set_title(weather_attribute)
            plt.show()

In [ ]:
for weather_attribute in weather.columns:
            fig = plt.figure(figsize=(10, 7))
            ax = fig.gca()
            
            weather["2019-12-16 00:00:00" : "2019-12-30 00:00:00"][weather_attribute].plot(ylabel=weather_attribute,xlabel='Day Index',grid='True', marker=".", figsize=(18, 6))
            #ax.set_xlabel(xlabel='%time')
            #ax.set_ylabel(ylabel=weather_attribute)
            ax.set_title(weather_attribute)
            plt.show()

In [ ]:
#from matplotlib import pyplot as plt
## 'Water_Quantity', 'Duration_of_Irrigation','Irrigation_Time_Intervals
#for col in green_house_climate.columns:
#    green_house_climate.boxplot(column=col, by='Duration_of_Irrigation', figsize=(6,6))
#    plt.title(col)
#plt.show()

In [ ]:
#find the normal difference between the set point of a feature and the measured value from sensors
x=green_house_climate['assim_sp']-green_house_climate['assim_vip']
x.mean()/green_house_climate['assim_sp'].mean()

In [ ]:
x=green_house_climate['co2_sp']-green_house_climate['co2_vip']
x.mean()/green_house_climate['co2_sp'].mean()

In [ ]:
sp_vip_paired_attributtes=[ [('assim_sp','assim_vip'),('assim_vip','assim_sp')],
                            [('co2_sp','co2_vip'),('co2_vip','co2_sp')],
                            [('dx_sp','dx_vip'),('dx_vip','dx_sp')],
                            [('int_blue_sp','int_blue_vip'),('int_blue_vip','int_blue_sp')],
                            [('int_farred_sp','int_farred_vip'),('int_farred_vip','int_farred_sp')],
                            [('int_red_sp','int_red_vip'),('int_red_vip','int_red_sp')], 
                            [('int_white_sp','int_white_vip'),('int_white_vip','int_white_sp')],
                            [('scr_blck_sp','scr_blck_vip'),('scr_blck_vip','scr_blck_sp')],
                            [('scr_enrg_sp','scr_enrg_vip'),('scr_enrg_vip','scr_enrg_sp')],
                            [('t_grow_min_sp','t_grow_min_vip'),('t_grow_min_vip','t_grow_min_sp')],
                            [('t_heat_sp','t_heat_vip'),('t_heat_vip','t_heat_sp')],
                            [('t_rail_min_sp','t_rail_min_vip'),('t_rail_min_vip','t_rail_min_sp')],
                            [('t_vent_sp','t_ventlee_vip'),('t_ventlee_vip','t_vent_sp')],
                            [('water_sup_intervals_sp_min','Irrigation_Time_Intervals'),('Irrigation_Time_Intervals','water_sup_intervals_sp_min')],
                            [('window_pos_lee_sp','window_pos_lee_vip'),('window_pos_lee_vip','window_pos_lee_sp')],
                            ]
norm_mean_difference=[]
similarity=[]

In [ ]:
for sp_vip_paired_attributte in sp_vip_paired_attributtes: 
    for (sp, vip) in sp_vip_paired_attributte:
        x=green_house_climate.iloc[(np.where(green_house_climate[sp].isnull()))][vip]
        #print(green_house_climate[sp])
        difference=(green_house_climate[sp]-green_house_climate[vip]).mean()
        mean_difference=difference/(green_house_climate[sp].mean())
        norm_mean_difference.append([sp,vip,mean_difference])
        
        #print(f'{sp},{vip} difference={norm_mean_difference}')
        #print(x.index)
        for i in x.index:
            #print(i)
            green_house_climate.loc[i,sp]= green_house_climate.loc[i,vip]#.iloc[i][vip]
            #print(i,green_house_climate.loc[i,sp]) 
        plt.figure()
        correlation = green_house_climate.corr()[vip][sp]
        corr1=green_house_climate.corr()[vip].sort_values(ascending=False)
        corr1.plot(kind='bar',title=f'{vip} SP-VIP Correlation',figsize=(18, 6),fontsize=12)
        plt.show
        save_fig(f'{vip}SP-VIP Correlation')
        print(f"correlation between {vip} and {sp}=",correlation)
        plt.show
norm_mean_difference     




In [ ]:
norm_diff=pd.DataFrame(np.array(norm_mean_difference),columns=['sp-vip','vip-sp','Norm difference'])


In [ ]:

green_house_climate.isnull().sum()

In [ ]:
#x=green_house_climate.iloc[(np.where(green_house_climate['assim_sp'].isnull()))]['assim_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'assim_sp']= green_house_climate.iloc[i]['assim_vip']
#        #print(i,green_house_climate.loc[i,'assim_sp'])  
#        
##green_house_climate.iloc[np.where(green_house_climate['assim_sp'].isnull())]['assim_sp']  =    green_house_climate.iloc[np.where(green_house_climate['assim_sp'].isnull())]['assim_vip']                 #785 
#x=green_house_climate.iloc[(np.where(green_house_climate['assim_vip'].isnull()))]['assim_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'assim_vip']= green_house_climate.iloc[i]['assim_sp']
#        #print(i,green_house_climate.loc[i,'assim_vip'])  
#
#
#
##green_house_climate['assim_vip']                        #71   
#        
#x=green_house_climate.iloc[(np.where(green_house_climate['co2_sp'].isnull()))]['co2_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'co2_sp']= green_house_climate.iloc[i]['co2_vip']
#        #print(i,green_house_climate.loc[i,'co2_sp']) 
##green_house_climate['co2_dos']                        #2    
##green_house_climate.iloc[np.where(green_house_climate['co2_sp'].isnull())]['co2_sp']  =    green_house_climate.iloc[np.where(green_house_climate['co2_sp'].isnull())]['co2_vip']                        #754
#
#x=green_house_climate.iloc[(np.where(green_house_climate['co2_vip'].isnull()))]['co2_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'co2_vip']= green_house_climate.iloc[i]['co2_sp']
#        #print(i,green_house_climate.loc[i,'co2_vip']) 
#
#
##green_house_climate['co2_vip']                       #71   
#
#x=green_house_climate.iloc[(np.where(green_house_climate['dx_sp'].isnull()))]['dx_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'dx_sp']= green_house_climate.iloc[i]['dx_vip']
#        #print(i,green_house_climate.loc[i,'dx_sp'])  
#        
#
#
##green_house_climate.iloc[np.where(green_house_climate['dx_sp'].isnull())]['dx_sp']  =    green_house_climate.iloc[np.where(green_house_climate['dx_sp'].isnull())]['dx_vip']                       #718  
#        
#x=green_house_climate.iloc[(np.where(green_house_climate['dx_vip'].isnull()))]['dx_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'dx_vip']= green_house_climate.iloc[i]['dx_sp']
#        #print(i,green_house_climate.loc[i,'dx_vip'])  
#        
#
##green_house_climate['dx_vip']                       #71   
#        
#x=green_house_climate.iloc[(np.where(green_house_climate['int_blue_sp'].isnull()))]['int_blue_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'int_blue_sp']= green_house_climate.iloc[i]['int_blue_vip']
#        #print(i,green_house_climate.loc[i,'int_blue_sp'])  
#        
#
##green_house_climate['int_blue_sp']                       #173  
#
#
#x=green_house_climate.iloc[(np.where(green_house_climate['int_blue_vip'].isnull()))]['int_blue_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'int_blue_vip']= green_house_climate.iloc[i]['int_blue_sp']
#        #print(i,green_house_climate.loc[i,'int_blue_vip'])  
#        
#
#
##green_house_climate.iloc[np.where(green_house_climate['int_blue_vip'].isnull())]['int_blue_vip']  =    green_house_climate.iloc[np.where(green_house_climate['int_blue_vip'].isnull())]['int_blue_sp']                   #22152
#
#x=green_house_climate.iloc[(np.where(green_house_climate['int_farred_vip'].isnull()))]['int_farred_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'int_farred_vip']= green_house_climate.iloc[i]['int_farred_sp']
#        #print(i,green_house_climate.loc[i,'int_farred_vip'])  
#
##green_house_climate['int_farred_sp']                   #14   
#
#x=green_house_climate.iloc[(np.where(green_house_climate['int_farred_sp'].isnull()))]['int_farred_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'int_farred_sp']= green_house_climate.iloc[i]['int_farred_vip']
#        #print(i,green_house_climate.loc[i,'int_farred_sp']) 
#
##green_house_climate.iloc[np.where(green_house_climate['int_farred_vip'].isnull())]['int_farred_vip']  =    green_house_climate.iloc[np.where(green_house_climate['int_farred_vip'].isnull())]['int_farred_sp']                   #22152
#
#x=green_house_climate.iloc[(np.where(green_house_climate['int_red_vip'].isnull()))]['int_red_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'int_red_vip']= green_house_climate.iloc[i]['int_red_sp']
#        #print(i,green_house_climate.loc[i,'int_red_vip'])  
#
#
#
##green_house_climate['int_red_sp']                   #173  
#
#x=green_house_climate.iloc[(np.where(green_house_climate['int_red_sp'].isnull()))]['int_red_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'int_red_sp']= green_house_climate.iloc[i]['int_red_vip']
#        #print(i,green_house_climate.loc[i,'int_red_sp'])  
#
#
#
##green_house_climate.iloc[np.where(green_house_climate['int_red_vip'].isnull())]['int_red_vip']  =    green_house_climate.iloc[np.where(green_house_climate['int_red_vip'].isnull())]['int_red_sp']                   #22152
#
#x=green_house_climate.iloc[(np.where(green_house_climate['int_white_vip'].isnull()))]['int_white_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'int_white_vip']= green_house_climate.iloc[i]['int_white_sp']
#        #print(i,green_house_climate.loc[i,'int_white_vip'])  
#
#
##green_house_climate['int_white_sp']                   #14   
#        
#x=green_house_climate.iloc[(np.where(green_house_climate['int_white_sp'].isnull()))]['int_white_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'int_white_sp']= green_house_climate.iloc[i]['int_white_vip']
#        #print(i,green_house_climate.loc[i,'int_white_sp'])  
#
#
##green_house_climate.iloc[np.where(green_house_climate['int_white_vip'].isnull())]['int_white_vip']  =    green_house_climate.iloc[np.where(green_house_climate['int_white_vip'].isnull())]['int_white_sp']                   #22152
##green_house_climate['pH_drain_PC']                   #71   
#
#x=green_house_climate.iloc[(np.where(green_house_climate['scr_blck_sp'].isnull()))]['scr_blck_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'scr_blck_sp']= green_house_climate.iloc[i]['scr_blck_vip']
#        #print(i,green_house_climate.loc[i,'scr_blck_sp'])  
#
#
#
##green_house_climate.iloc[np.where(green_house_climate['scr_blck_sp'].isnull())]['scr_blck_sp']  =    green_house_climate.iloc[np.where(green_house_climate['scr_blck_sp'].isnull())]['scr_blck_vip']                   #783  
#
#x=green_house_climate.iloc[(np.where(green_house_climate['scr_blck_vip'].isnull()))]['scr_blck_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'scr_blck_vip']= green_house_climate.iloc[i]['scr_blck_sp']
#        #print(i,green_house_climate.loc[i,'scr_blck_vip'])  
#
#
#
##green_house_climate['scr_blck_vip']                   #71   
#
#x=green_house_climate.iloc[(np.where(green_house_climate['scr_enrg_sp'].isnull()))]['scr_enrg_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'scr_enrg_sp']= green_house_climate.iloc[i]['scr_enrg_vip']
#        #print(i,green_house_climate.loc[i,'scr_enrg_sp'])  
#
#
##green_house_climate.iloc[np.where(green_house_climate['scr_enrg_sp'].isnull())]['scr_enrg_sp']  =    green_house_climate.iloc[np.where(green_house_climate['scr_enrg_sp'].isnull())]['scr_enrg_vip']                  #785  
#
#x=green_house_climate.iloc[(np.where(green_house_climate['scr_enrg_vip'].isnull()))]['scr_enrg_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'scr_enrg_vip']= green_house_climate.iloc[i]['scr_enrg_sp']
#        #print(i,green_house_climate.loc[i,'scr_enrg_vip'])  
#
#
#
##green_house_climate['scr_enrg_vip']                   #71   
#
#
#x=green_house_climate.iloc[(np.where(green_house_climate['t_grow_min_sp'].isnull()))]['t_grow_min_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'t_grow_min_sp']= green_house_climate.iloc[i]['t_grow_min_vip']
#        #print(i,green_house_climate.loc[i,'t_grow_min_sp'])  
#
#
##green_house_climate.iloc[np.where(green_house_climate['t_grow_min_sp'].isnull())]['t_grow_min_sp']  =    green_house_climate.iloc[np.where(green_house_climate['t_grow_min_sp'].isnull())]['t_grow_min_vip']                  #1505 
#
#x=green_house_climate.iloc[(np.where(green_house_climate['t_grow_min_vip'].isnull()))]['t_grow_min_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'t_grow_min_vip']= green_house_climate.iloc[i]['t_grow_min_sp']
#        #print(i,green_house_climate.loc[i,'t_grow_min_vip'])  
#
##green_house_climate['t_grow_min_vip']                   #71   
#
#x=green_house_climate.iloc[(np.where(green_house_climate['t_heat_sp'].isnull()))]['t_heat_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'t_heat_sp']= green_house_climate.iloc[i]['t_heat_vip']
#        #print(i,green_house_climate.loc[i,'t_heat_sp'])  
#
#
##green_house_climate.iloc[np.where(green_house_climate['t_heat_sp'].isnull())]['t_heat_sp']  =    green_house_climate.iloc[np.where(green_house_climate['t_heat_sp'].isnull())]['t_heat_vip']                   #706  
#
#x=green_house_climate.iloc[(np.where(green_house_climate['t_heat_vip'].isnull()))]['t_heat_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'t_heat_vip']= green_house_climate.iloc[i]['t_heat_sp']
#        #print(i,green_house_climate.loc[i,'t_heat_vip'])  
#
##green_house_climate['t_heat_vip']                   #71   
#
#
#x=green_house_climate.iloc[(np.where(green_house_climate['t_rail_min_sp'].isnull()))]['t_rail_min_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'t_rail_min_sp']= green_house_climate.iloc[i]['t_rail_min_vip']
#        #print(i,green_house_climate.loc[i,'t_rail_min_sp'])  
#
#
#
##green_house_climate.iloc[np.where(green_house_climate['t_rail_min_sp'].isnull())]['t_rail_min_sp']  =    green_house_climate.iloc[np.where(green_house_climate['t_rail_min_sp'].isnull())]['t_rail_min_vip']                   #787  
#x=green_house_climate.iloc[(np.where(green_house_climate['t_rail_min_vip'].isnull()))]['t_rail_min_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'t_rail_min_vip']= green_house_climate.iloc[i]['t_rail_min_sp']
#        #print(i,green_house_climate.loc[i,'t_rail_min_vip'])  
#
#
##green_house_climate['t_rail_min_vip']                   #71   
#
#x=green_house_climate.iloc[(np.where(green_house_climate['t_vent_sp'].isnull()))]['t_vent_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'t_vent_sp']= green_house_climate.iloc[i]['t_ventlee_vip']
#        #print(i,green_house_climate.loc[i,'t_vent_sp'])  
#
#
##green_house_climate.iloc[np.where(green_house_climate['t_vent_sp'].isnull())]['t_vent_sp']  =    green_house_climate.iloc[np.where(green_house_climate['t_vent_sp'].isnull())]['t_ventlee_vip']                   #12576
#
#x=green_house_climate.iloc[(np.where(green_house_climate['t_ventlee_vip'].isnull()))]['t_ventlee_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'t_ventlee_vip']= green_house_climate.iloc[i]['t_vent_sp']
#        #print(i,green_house_climate.loc[i,'t_ventlee_vip'])  
#
#
#
##green_house_climate['t_ventlee_vip']                   #71   
##green_house_climate['t_ventwind_vip']                   #71   
#
#x=green_house_climate.iloc[(np.where(green_house_climate['water_sup_intervals_sp_min'].isnull()))]['water_sup_intervals_sp_min']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'water_sup_intervals_sp_min']= green_house_climate.iloc[i]['Irrigation_Time_Intervals']
#        #print(i,green_house_climate.loc[i,'water_sup_intervals_sp_min'])  
#
##green_house_climate.iloc[np.where(green_house_climate['water_sup_intervals_sp_min'].isnull())]['water_sup_intervals_sp_min']  =    green_house_climate.iloc[np.where(green_house_climate['water_sup_intervals_sp_min'].isnull())]['Irrigation_Time_Intervals']    #1410 
#
#x=green_house_climate.iloc[(np.where(green_house_climate['window_pos_lee_sp'].isnull()))]['window_pos_lee_sp']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'window_pos_lee_sp']= green_house_climate.iloc[i]['window_pos_lee_vip']
#        #print(i,green_house_climate.loc[i,'window_pos_lee_sp'])  
#
#
##green_house_climate.iloc[np.where(green_house_climate['window_pos_lee_sp'].isnull())]['window_pos_lee_sp']  =    green_house_climate.iloc[np.where(green_house_climate['window_pos_lee_sp'].isnull())]['window_pos_lee_vip']    #715  
#x=green_house_climate.iloc[(np.where(green_house_climate['window_pos_lee_vip'].isnull()))]['window_pos_lee_vip']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'window_pos_lee_vip']= green_house_climate.iloc[i]['window_pos_lee_sp']
#        #print(i,green_house_climate.loc[i,'window_pos_lee_vip'])  
#
##green_house_climate['window_pos_lee_vip']    #71   
##green_house_climate['Water_Quantity']    #71   
##green_house_climate['Duration_of_Irrigation']    #71   
##green_house_climate['Irrigation_Time_Intervals']    #71   
#x=green_house_climate.iloc[(np.where(green_house_climate['Irrigation_Time_Intervals'].isnull()))]['Irrigation_Time_Intervals']
#for i in x.index:
#        #print(i)
#        green_house_climate.loc[i,'Irrigation_Time_Intervals']= green_house_climate.iloc[i]['water_sup_intervals_sp_min']
#        #print(i,green_house_climate.loc[i,'Irrigation_Time_Intervals'])  
#
#

In [ ]:
green_house_climate.isnull().sum()

In [ ]:
from datetime import datetime
for gh_attribute in green_house_climate.columns:
    x=green_house_climate.iloc[(np.where(green_house_climate[gh_attribute].isnull()))][gh_attribute]
    for i in reversed(x.index):
        if i.day+(1)< green_house_climate.index[-1].day:
            #print(i+(12*24))
            d=i
            print(d)
            
            d = d.replace(day=d.day+1)
           
            print(d)
            green_house_climate.loc[i,gh_attribute]= green_house_climate.loc[d,gh_attribute]
            #print(i+(12*24),green_house_climate.loc[i,gh_attribute])
for gh_attribute in green_house_climate.columns:
    x=green_house_climate.iloc[(np.where(green_house_climate[gh_attribute].isnull()))][gh_attribute]
    for i in x.index:
        if i.day-(1)>= 0:
            #print(i+(12*24))
            d=i
            print(d)
            d = d.replace(day=d.day-1)
            print(d)
            green_house_climate.loc[i,gh_attribute]= green_house_climate.loc[d,gh_attribute]
            #print(i+(12*24),green_house_climate.loc[i,gh_attribute])

In [ ]:
green_house_climate.isnull().sum()

In [ ]:
#green_house_climate = green_house_climate.set_index("%time")


In [ ]:
green_house_climate.shape[1]

In [ ]:
root_zone=pd.read_csv("C:\\Users\\user\\Desktop\\iman\\Master\\RoboticsAndControlSystems\\Project\\Data\\Automatoes\\GrodanSens.csv",parse_dates=["%time"],low_memory=False).set_index('%time').astype(float)


In [ ]:
root_zone.info()

In [ ]:
root_zone.head()

In [ ]:
root_zone.isnull().sum()

In [ ]:
x=root_zone.iloc[(np.where(root_zone['EC_slab1'].isnull()))]


In [ ]:
#for root_attribute in ['EC_slab1'  , 'EC_slab2' , 'WC_slab1' , 'WC_slab2'   , 't_slab1','t_slab2' ]:
#    for i in reversed(x.index):
#        if i.day+1< int(root_zone.index[-1].day):
#            #print(i+(12*24))
#            root_zone.loc[i,root_attribute]= root_zone.iloc[i+12*24][root_attribute]
#            #print(i+(12*24),root_zone.loc[i,root_attribute])


for rz_attribute in root_zone.columns:
    x=root_zone.iloc[(np.where(root_zone[rz_attribute].isnull()))][rz_attribute]
    for i in reversed(x.index):
        if i.day+(1)< root_zone.index[-1].day:
            #print(i+(12*24))
            d=i
            print(d)
            
            d = d.replace(day=d.day+1)
           
            print(d)
            root_zone.loc[i,rz_attribute]= root_zone.loc[d,rz_attribute]
            #print(i+(12*24),root_zone.loc[i,rz_attribute])

            

In [ ]:
#for root_attribute in ['EC_slab1'  , 'EC_slab2' , 'WC_slab1' , 'WC_slab2'   , 't_slab1','t_slab2' ]:
#    for i in x.index:
#        if i-(12*24)>= 0:
#            #print(i+(12*24))
#            root_zone.loc[i,root_attribute]= root_zone.iloc[i-12*24][root_attribute]
#            #print(i+(12*24),root_zone.loc[i,root_attribute])

for rz_attribute in root_zone.columns:
    x=root_zone.iloc[(np.where(root_zone[rz_attribute].isnull()))][rz_attribute]
    for i in x.index:
        if i.day-(1)>= root_zone.index[0].day:
            #print(i+(12*24))
            d=i
            print(d)
            
            d = d.replace(day=d.day-1)
           
            print(d)
            root_zone.loc[i,rz_attribute]= root_zone.loc[d,rz_attribute]
            #print(i+(12*24),root_zone.loc[i,rz_attribute])

            



In [ ]:
root_zone.isnull().sum()

In [ ]:
#root_zone = root_zone.sort_values("%time").set_index("%time")


In [ ]:
root_zone.hist(bins=50,figsize=(20,15),column=root_zone.columns)

In [ ]:
root_zone.index.duplicated().sum()

In [ ]:
green_house_climate.index.duplicated().sum()

In [ ]:
green_house=pd.concat([weather,root_zone,green_house_climate],axis=1)

In [ ]:
weather.columns.shape+green_house_climate.columns.shape+root_zone.columns.shape

In [ ]:
green_house.shape

In [ ]:
x=np.concatenate([weather.columns.tolist(),root_zone.columns.tolist(),green_house_climate.columns.tolist()])


In [ ]:
pd.set_option('display.max_rows',43000)
print(green_house.isnull().sum().sum())

In [ ]:
green_house=pd.DataFrame(green_house,columns=np.concatenate([weather.columns.tolist(),root_zone.columns.tolist(),green_house_climate.columns.tolist()]))

In [ ]:
green_house.shape

In [ ]:
#one_day = 12 * 24
#for row in range(green_house.shape[0]):
#	for col in range(green_house.shape[1]):
#		if np.isnan(green_house.iloc[row][col]):
#			green_house.iloc[row][col] = green_house.iloc[row - one_day][col]

In [ ]:
#green_house.iloc[np.where(green_house['EC_slab1'].isnull())]['EC_slab1'] = green_house.iloc[np.where(green_house['EC_slab1'].isnull())]['EC_slab1']

In [ ]:
import matplotlib.pyplot as plt

for num_attribute in green_house.columns:
    
    green_house["2019-12-16 00:00:00" : "2019-12-25 00:00:00"][num_attribute].plot(ylabel=num_attribute,grid=True, marker=".", figsize=(18, 6))
    save_fig(f'green_house_{num_attribute}_time_series_plot')

    plt.show()

In [ ]:

#find a trend 
#green_house['time']=green_house.index
##print(green_house['%time'].astype('string').str.split('.',expand=True).astype('int')[0])
#x=green_house['time'].astype('string').str.split('.',expand=True).astype('int')[0]
#green_house['time']=x
#print('green_house[time]',green_house.index)
#print('************')
for green_house_attribute in green_house.columns:
            fig = plt.figure(figsize=(10, 7))
            ax = fig.gca()
            
            dayly_avg=green_house[green_house_attribute].resample('H').mean()
            period=slice("2019-12-16 00:00:00", "2020-05-30 00:00:00")
            #print(green_house_attribute,green_house.groupby(by=['time'])[green_house_attribute].mean())
            green_house[green_house_attribute].plot(ax=ax,ylabel=green_house_attribute,xlabel='Day Index',grid='True', marker=".", figsize=(18, 6))
            
            rolling_average_dayly = dayly_avg[period].rolling(window=24*10).mean()
            #print(green_house_attribute,rolling_average_dayly)
            rolling_average_dayly.plot(grid=True)
            ax.set_xlabel(xlabel='%time')
            ax.set_ylabel(ylabel=green_house_attribute)
            ax.set_title(green_house_attribute)
            plt.show()



In [ ]:
green_house['%time']=green_house.index
green_house

In [ ]:
#green_house['%time']=green_house.index
#out=green_house['%time'].astype('string').str.split('.',expand=True).reset_index()
#out
#out['days']=np.zeros(shape=out[0].shape[0])
#out['hours']=np.zeros(shape=out[0].shape[0])
#out['minutes']=np.zeros(shape=out[0].shape[0])
#
#out.shape

In [ ]:
#out=np.load('C:\\Users\\user\\Desktop\\iman\\Master\\RoboticsAndControlSystems\\Project\\Documentation\\out.np.npy',allow_pickle=True)

In [ ]:
green_house['years'] =   green_house['%time'].dt.year
green_house['months'] =  green_house['%time'].dt.month
green_house['days'] =    green_house['%time'].dt.day
green_house['hours'] =   green_house['%time'].dt.hour
green_house['minutes'] = green_house['%time'].dt.minute
green_house.head()


In [ ]:
#out

In [ ]:
#day_count=1
#five_min_counts=1
#for i in (out.groupby(by=[0])[0].sum().index):
#    #print(i)
#    five_min_countsmin_counts=1
#    hours_count=1
#    for idx in out[0].index:     
#        if out[0].iloc[idx]==i:
#            out['days'].iloc[idx]=day_count
#            #print('done',idx)
#            out['hours'].iloc[idx]=hours_count
#            out['minutes'].iloc[idx]=five_min_counts*5
#            five_min_counts+=1
#            if five_min_counts>12:
#                hours_count+=1
#                five_min_counts=1
#    day_count+=1
#out

In [ ]:
#day_count=1
#min_counts=1
#for i in (out.groupby(by=[0])[0].sum().index):
#    #print(i)
#    min_counts=1
#    hours_count=1
#    for idx in out[0].index:     
#        if out[0].iloc[idx]==i:
#            out['days'].iloc[idx]=day_count
#            #print('done',idx)
#            out['hours'].iloc[idx]=hours_count
#            min_counts+=1
#            if min_counts>12:
#                hours_count+=1
#                min_counts=1
#    day_count+=1
#out

In [ ]:
print(green_house['days'].value_counts())

In [ ]:
print(green_house['minutes'].value_counts())

In [ ]:
print(green_house['hours'].value_counts())

In [ ]:
green_house['days'].values
green_house['hours'].values
green_house['minutes'].values

#green_house.drop('%time',inplace=True)
#green_house['hours']

In [ ]:
#green_house.drop(columns=['minutes'],inplace=True)

In [ ]:
#green_house.isnull().sum()


In [ ]:
import matplotlib.pyplot as plt

for num_attribute in green_house.columns:
    
    green_house["2019-12-16 00:00:00" : "2020-12-26 00:00:00"][num_attribute].plot(ylabel=num_attribute,xlabel='Day Index',grid=True, marker=".", figsize=(18, 6),title=num_attribute)
    
    save_fig(f'green_house_{num_attribute}_time_series_plot')

    plt.show()

In [ ]:
green_house[green_house.columns].corr()['Irrigation_Time_Intervals'].sort_values(ascending=False)


In [ ]:
green_house[green_house.columns].corr()['BlackScr'	]


In [ ]:
corr0=green_house[green_house.columns].corr()['Water_Quantity'].sort_values(ascending=False)
corr0.drop(['Irrigation_Time_Intervals','Water_Quantity','Duration_of_Irrigation'],axis=0,inplace=True)
corr0.plot(kind='bar',title='Water_Quantity Correlation',figsize=(18, 6),fontsize=12)
save_fig('Water_Quantity Correlation with features')


In [ ]:
corr1=green_house[green_house.columns].corr()['Irrigation_Time_Intervals'].sort_values(ascending=False)
corr1.drop(['Irrigation_Time_Intervals','Water_Quantity','Duration_of_Irrigation'],axis=0,inplace=True)

In [ ]:
corr1.plot(kind='bar',title='Irrigation_Time_Intervals Correlation',figsize=(18, 6),fontsize=12)
save_fig('Irrigation_Time_Intervals Correlation with features')


In [ ]:
corr2=green_house[green_house.columns].corr()['Duration_of_Irrigation'].sort_values(ascending=False)
corr2.drop(['Irrigation_Time_Intervals','Water_Quantity','Duration_of_Irrigation'],axis=0,inplace=True)
corr2.plot(kind='bar',title='Duration_of_Irrigation Correlation',figsize=(18, 6),fontsize=12)
save_fig(f'Duration_of_Irrigation Correlation with features')


In [ ]:
correlation_values=pd.concat([corr0,corr1,corr2],axis=1)

In [ ]:
correlation_values
headers = {
    "selector": "th:not(.index_name)",
    "props": "background-color: #800000; color: white; text-align: center"
}
properties = {"border": "1px solid black", "width": "65px", "text-align": "center"}
cell_hover = {
    "selector": "td:hover",
    "props": [("background-color", "#FFFFE0")]
}
index_names = {
    "selector": ".index_name",
    "props": "font-style: italic; color: darkgrey; font-weight:normal;"
}
correlation_values.style.format(precision=2).set_table_styles([cell_hover, index_names, headers]).set_properties(**properties)
correlation_values.style.highlight_max(color="red",axis=0)
correlation_values.style.highlight_min(color="red",axis=0)

def mean_highlighter(x):
    style_lt = "background-color: whight"
    style_gt = "background-color: red"
    gt_mean = abs(x) > .4
    return [style_gt if i else style_lt for i in gt_mean]
    
    
correlation_values.style.apply(mean_highlighter)

In [ ]:
np.save('green_house.npy',green_house)
#print(out['hours'].value_counts())
#print(out['days'].value_counts())
#out=np.load('out.npy')
green_house.to_csv('C:\\Users\\user\\Desktop\\iman\\AGHC\\AutonomousGreenHouseChallenge\\Code\\Training\\greenhouse_fill_missing_values.csv')
weather.to_csv('C:\\Users\\user\\Desktop\\iman\\AGHC\\AutonomousGreenHouseChallenge\\Code\\Training\\weather_fill_missing_values.csv')
green_house_climate.to_csv('C:\\Users\\user\\Desktop\\iman\\AGHC\\AutonomousGreenHouseChallenge\\Code\\Training\\greenhouse_climate_fill_missing_values.csv')
root_zone.to_csv('C:\\Users\\user\\Desktop\\iman\\AGHC\\AutonomousGreenHouseChallenge\\Code\\Training\\root_zone_fill_missing_values.csv')


In [ ]:
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR 
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score,cross_val_predict
import numpy as np
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor
from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.preprocessing import FunctionTransformer


In [ ]:
index=green_house[green_house['Irrigation_Time_Intervals'].isnull()==True].index
green_house.drop(index=index,inplace=True)

In [ ]:
y=pd.DataFrame(green_house.loc[:,['Water_Quantity','Duration_of_Irrigation','Irrigation_Time_Intervals']])

X=green_house.drop(['Water_Quantity','Duration_of_Irrigation','Irrigation_Time_Intervals'],axis=1).copy()
print(y.shape)
print(X.shape)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0,shuffle=True,
                                                    stratify=green_house['Rain'])


#from sklearn.preprocessing import MinMaxScaler
#min_max_scaler=MinMaxScaler()
#print(X_train.shape)
#y_train=min_max_scaler.fit_transform(y_train)
#y_test=min_max_scaler.transform(y_test)
#print('y_train',y_train.shape)
#print('y_test',y_test.shape)
#

In [ ]:
y_train.mean(axis=0)
y_train.std(axis=0)

#11.551732063293457,
# 2.028500556945801,
# 211.20957946777344

In [ ]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

In [ ]:
#from sklearn.metrics import mean_squared_error,r2_score
#from sklearn.model_selection import cross_val_score,cross_val_predict,cross_validate
#Water_Quantity_train_results=[] 
#Irrigation_Time_Intervals_train_results=[]
#Duration_of_Irrigation_train_results=[]
#Water_Quantity_test_results=[] 
#Irrigation_Time_Intervals_test_results=[]
#Duration_of_Irrigation_test_results=[]
#train_rmse=dict()
#train_r2=dict()
#train_mse=dict()
#test_rmse=dict()
#test_r2=dict()
#test_mse=dict()
#confidence_intervals=[]
#for key in models.keys():
#    full_pipeline = Pipeline([
#        ('preprocessing', preprocessing),
#        ('models',models[key]),
#    ])
#    #train
#    score=cross_validate(full_pipeline, X_train, y_train,scoring='neg_root_mean_squared_error', cv=10)
#    #print(score)
#    #train_rmse=-score['test_score'].mean()/y.mean()
#    #print(train_rmse)
#    train_prediction=cross_val_predict(full_pipeline,X_train,y_train,cv=10)
#    test_prediction=cross_val_predict(full_pipeline,X_test,y_test,cv=10)
#
#    for i,out in zip(range(0,len(y.columns),1),y.columns):
#        #train scores
#        train_rmse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train[:,i],squared=False)#/y[out].mean()
#        train_mse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train[:,i])#/y[out].mean()
#        train_r2[out]=r2_score(y_true=y_train[:,i],y_pred=train_prediction[:,i])
#
#        #test scores
#        test_rmse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test[:,i],squared=False)#/y[out].mean()
#        test_mse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test[:,i])#/y[out].mean()
#        test_r2[out]=r2_score(y_true=y_test[:,i],y_pred=test_prediction[:,i])
#
#    #train result for each output
#    Water_Quantity_train_results.append((key,train_rmse['Water_Quantity'],train_mse['Water_Quantity'],train_r2['Water_Quantity']))
#    Duration_of_Irrigation_train_results.append((key,train_rmse['Duration_of_Irrigation'],train_mse['Duration_of_Irrigation'],train_r2['Duration_of_Irrigation']))
#    Irrigation_Time_Intervals_train_results.append((key,train_rmse['Irrigation_Time_Intervals'],train_mse['Irrigation_Time_Intervals'],train_r2['Irrigation_Time_Intervals']))
#    #test result for each output
#    Water_Quantity_test_results.append((key,test_rmse['Water_Quantity'],test_mse['Water_Quantity'],test_r2['Water_Quantity']))
#    Duration_of_Irrigation_test_results.append((key,test_rmse['Duration_of_Irrigation'],test_mse['Duration_of_Irrigation'],test_r2['Duration_of_Irrigation']))
#    Irrigation_Time_Intervals_test_results.append((key,test_rmse['Irrigation_Time_Intervals'],test_mse['Irrigation_Time_Intervals'],test_r2['Irrigation_Time_Intervals']))
#   
#print(Water_Quantity_train_results,'\n',Duration_of_Irrigation_train_results,'\n',Irrigation_Time_Intervals_train_results,'\n')
#print(Water_Quantity_test_results,'\n',Duration_of_Irrigation_test_results,'\n',Irrigation_Time_Intervals_test_results,'\n')

In [ ]:
##show training results table:
#print(Water_Quantity_train_results,Duration_of_Irrigation_train_results,Irrigation_Time_Intervals_train_results)
#
#print('models train Water_Quantity scores:',Water_Quantity_train_results)
#best_Water_Quantity_trained_model_idx=np.array(Water_Quantity_train_results)[:,1].argmin()
#
#print('models train Duration_of_Irrigation scores:',Duration_of_Irrigation_train_results)
#best_Duration_of_Irrigation_trained_model_idx=np.array(Duration_of_Irrigation_train_results)[:,1].argmin()
#
#print('models train Water_interv scores:',Irrigation_Time_Intervals_train_results)
#best_Irrigation_Time_Intervals_trained_model_idx=np.array(Irrigation_Time_Intervals_train_results)[:,1].argmin()
#
##print('models test scores:',test_results)
##best_tested_model_idx=np.array(test_results)[:,1].argmax()
#
#
#print('best trained model:',Water_Quantity_train_results[best_Water_Quantity_trained_model_idx][0],Water_Quantity_train_results[best_Water_Quantity_trained_model_idx][1],'\n',
#                            Duration_of_Irrigation_train_results[best_Duration_of_Irrigation_trained_model_idx][0],Duration_of_Irrigation_train_results[best_Duration_of_Irrigation_trained_model_idx][1],'\n',
#                            Irrigation_Time_Intervals_train_results[best_Irrigation_Time_Intervals_trained_model_idx][0],Irrigation_Time_Intervals_train_results[best_Irrigation_Time_Intervals_trained_model_idx][1],'\n',
#)
#
##print('best tested model:',test_results[best_tested_model_idx][0],test_results[best_tested_model_idx][1])
#Water_Quantity_train_result=pd.DataFrame(Water_Quantity_train_results,
#           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
#
#Duration_of_Irrigation_train_result=pd.DataFrame(Duration_of_Irrigation_train_results,
#           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
#
#Irrigation_Time_Intervals_train_result=pd.DataFrame(Irrigation_Time_Intervals_train_results,
#           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
##import dataframe_image as dfi
#
##dfi.export(train_result, "ModelsTrainingResultsTable.png")
#Water_Quantity_train_result
#

In [ ]:
#Duration_of_Irrigation_train_result 

In [ ]:
#Irrigation_Time_Intervals_train_result

In [ ]:
#print(train_prediction,y_train)

In [ ]:
##show testing results table:
#print(Water_Quantity_test_results,Duration_of_Irrigation_test_results,Irrigation_Time_Intervals_test_results)
#
#print('models test Water_Quantity scores:',Water_Quantity_test_results)
#best_Water_Quantity_test_model_idx=np.array(Water_Quantity_test_results)[:,1].argmin()
#
#print('models test Duration_of_Irrigation scores:',Duration_of_Irrigation_test_results)
#best_Duration_of_Irrigation_test_model_idx=np.array(Duration_of_Irrigation_test_results)[:,1].argmin()
#
#print('models test Water_interv scores:',Irrigation_Time_Intervals_test_results)
#best_Irrigation_Time_Intervals_test_model_idx=np.array(Irrigation_Time_Intervals_test_results)[:,1].argmin()
#
##print('models test scores:',test_results)
##best_tested_model_idx=np.array(test_results)[:,1].argmax()
#
#
#print('best tested model:',Water_Quantity_test_results[best_Water_Quantity_test_model_idx][0],Water_Quantity_test_results[best_Water_Quantity_test_model_idx][1],'\n',
#                            Duration_of_Irrigation_test_results[best_Duration_of_Irrigation_test_model_idx][0],Duration_of_Irrigation_test_results[best_Duration_of_Irrigation_test_model_idx][1],'\n',
#                            Irrigation_Time_Intervals_test_results[best_Irrigation_Time_Intervals_test_model_idx][0],Irrigation_Time_Intervals_test_results[best_Irrigation_Time_Intervals_test_model_idx][1],'\n',
#)
#
#Water_Quantity_test_result=pd.DataFrame(Water_Quantity_test_results,
#           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
#
#Duration_of_Irrigation_test_result=pd.DataFrame(Duration_of_Irrigation_test_results,
#           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
#
#Irrigation_Time_Intervals_test_result=pd.DataFrame(Irrigation_Time_Intervals_test_results,
#           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
##import dataframe_image as dfi
#
##dfi.export(test_result, "ModelstestingResultsTable.png")
#Water_Quantity_test_result
#

In [ ]:
#Duration_of_Irrigation_test_result

In [ ]:
#Irrigation_Time_Intervals_test_result

In [ ]:
#from sklearn.multioutput import MultiOutputRegressor
#
#models={'LinearRegression':LinearRegression(),
#    #'GradientBoostingRegressor':GradientBoostingRegressor(random_state=42),
#    #'AdaBoostRegressor':AdaBoostRegressor(random_state=42),
#    #'Lasso':Lasso(random_state=42),
#    #'Ridge':Ridge(random_state=42),
#    #'ElasticNet':ElasticNet(random_state=42),
#    'KNeighborsRegressor':KNeighborsRegressor(),
#    'DecisionTreeRegressor':DecisionTreeRegressor(random_state=42),
#    'RandomForestRegressor':RandomForestRegressor(random_state=42),
#    #'LinearSVR':LinearSVR(max_iter=50000,random_state=42),
#    #'SVR':  SVR(max_iter=50000)
#    }
#
#num_attributes=['pH_drain_PC' ,'WC_slab1','Rhout' ,                        
#'WC_slab2','RadSum','Pyrgeo','EC_slab1','AbsHumOut','EC_slab2','t_slab1',                       
#'t_slab2','Winddir','HumDef','CO2air','Rhair',                         
#'Tout','EC_drain_PC','Tair','Windsp','co2_dos','Iglob','PARout','Tot_PAR', 'days','hours' ] 
#
#
#cat_attributes=['Rain']
#outputs=['Water_Quantity' , 'Duration_of_Irrigation' , 'Irrigation_Time_Intervals']
#
#num_pipline=make_pipeline(SimpleImputer(strategy='mean'),StandardScaler())
#cat_pipeline=make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder())
#
##date_pipeline=make_pipeline(FunctionTransformer(lambda x:pd.DatetimeIndex(x).day))
#preprocessing=ColumnTransformer([('num', num_pipline, num_attributes),
#                                ('cat',cat_pipeline,cat_attributes),
#                                #('day',date_pipeline,['dteday'])
#                                ])
#
##data_prepared=preprocessing.fit_transform()
##data_prepared.shape
#
#from sklearn.metrics import mean_squared_error,r2_score
#from sklearn.model_selection import cross_val_score,cross_val_predict,cross_validate
#Water_Quantity_train_results=[] 
#Irrigation_Time_Intervals_train_results=[]
#Duration_of_Irrigation_train_results=[]
#Water_Quantity_test_results=[] 
#Irrigation_Time_Intervals_test_results=[]
#Duration_of_Irrigation_test_results=[]
#train_rmse=dict()
#train_r2=dict()
#train_mse=dict()
#test_rmse=dict()
#test_r2=dict()
#test_mse=dict()
#confidence_intervals=[]
#
#for key in models.keys():
#    #models['MultiOutputRegressor'] = MultiOutputRegressor(models[key],n_jobs=-1)
#
#    full_pipeline = Pipeline([
#        ('preprocessing', preprocessing),
#        ('MultiOutputRegressor',MultiOutputRegressor(models[key],n_jobs=-1)),
#    ])
#    #train
#    score=cross_validate(full_pipeline, X_train, y_train,scoring='neg_root_mean_squared_error', cv=10)
#    #print(score)
#    #train_rmse=-score['test_score'].mean()/y.mean()
#    #print(train_rmse)
#    train_prediction=cross_val_predict(full_pipeline,X_train,y_train,cv=10)
#    test_prediction=cross_val_predict(full_pipeline,X_test,y_test,cv=10)
#
#    for i,out in zip(range(0,len(y.columns),1),y.columns):
#        #train scores
#        train_rmse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train[:,i],squared=False)#/y[out].mean()
#        train_mse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train[:,i])#/y[out].mean()
#        train_r2[out]=r2_score(y_true=y_train[:,i],y_pred=train_prediction[:,i])
#
#        #test scores
#        test_rmse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test[:,i],squared=False)#/y[out].mean()
#        test_mse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test[:,i])#/y[out].mean()
#        test_r2[out]=r2_score(y_true=y_test[:,i],y_pred=test_prediction[:,i])
#
#    #train result for each output
#    Water_Quantity_train_results.append((key,train_rmse['Water_Quantity'],train_mse['Water_Quantity'],train_r2['Water_Quantity']))
#    Duration_of_Irrigation_train_results.append((key,train_rmse['Duration_of_Irrigation'],train_mse['Duration_of_Irrigation'],train_r2['Duration_of_Irrigation']))
#    Irrigation_Time_Intervals_train_results.append((key,train_rmse['Irrigation_Time_Intervals'],train_mse['Irrigation_Time_Intervals'],train_r2['Irrigation_Time_Intervals']))
#    #test result for each output
#    Water_Quantity_test_results.append((key,test_rmse['Water_Quantity'],test_mse['Water_Quantity'],test_r2['Water_Quantity']))
#    Duration_of_Irrigation_test_results.append((key,test_rmse['Duration_of_Irrigation'],test_mse['Duration_of_Irrigation'],test_r2['Duration_of_Irrigation']))
#    Irrigation_Time_Intervals_test_results.append((key,test_rmse['Irrigation_Time_Intervals'],test_mse['Irrigation_Time_Intervals'],test_r2['Irrigation_Time_Intervals']))
#   
#
#print(Water_Quantity_train_results,'\n',Duration_of_Irrigation_train_results,'\n',Irrigation_Time_Intervals_train_results,'\n')
#print(Water_Quantity_test_results,'\n',Duration_of_Irrigation_test_results,'\n',Irrigation_Time_Intervals_test_results,'\n')

In [ ]:
##show training results table:
#print(Water_Quantity_train_results,Duration_of_Irrigation_train_results,Irrigation_Time_Intervals_train_results)
#
#print('models train Water_Quantity scores:',Water_Quantity_train_results)
#best_Water_Quantity_trained_model_idx=np.array(Water_Quantity_train_results)[:,1].argmin()
#
#print('models train Duration_of_Irrigation scores:',Duration_of_Irrigation_train_results)
#best_Duration_of_Irrigation_trained_model_idx=np.array(Duration_of_Irrigation_train_results)[:,1].argmin()
#
#print('models train Water_interv scores:',Irrigation_Time_Intervals_train_results)
#best_Irrigation_Time_Intervals_trained_model_idx=np.array(Irrigation_Time_Intervals_train_results)[:,1].argmin()
#
##print('models test scores:',test_results)
##best_tested_model_idx=np.array(test_results)[:,1].argmax()
#
#
#print('best trained model:',Water_Quantity_train_results[best_Water_Quantity_trained_model_idx][0],Water_Quantity_train_results[best_Water_Quantity_trained_model_idx][1],'\n',
#                            Duration_of_Irrigation_train_results[best_Duration_of_Irrigation_trained_model_idx][0],Duration_of_Irrigation_train_results[best_Duration_of_Irrigation_trained_model_idx][1],'\n',
#                            Irrigation_Time_Intervals_train_results[best_Irrigation_Time_Intervals_trained_model_idx][0],Irrigation_Time_Intervals_train_results[best_Irrigation_Time_Intervals_trained_model_idx][1],'\n',
#)
#
##print('best tested model:',test_results[best_tested_model_idx][0],test_results[best_tested_model_idx][1])
#Water_Quantity_train_result=pd.DataFrame(Water_Quantity_train_results,
#           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
#
#Duration_of_Irrigation_train_result=pd.DataFrame(Duration_of_Irrigation_train_results,
#           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
#
#Irrigation_Time_Intervals_train_result=pd.DataFrame(Irrigation_Time_Intervals_train_results,
#           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
##import dataframe_image as dfi
#
##dfi.export(train_result, "ModelsTrainingResultsTable.png")
#Water_Quantity_train_result
#

In [ ]:
#Duration_of_Irrigation_train_result

In [ ]:
#Irrigation_Time_Intervals_train_result
green_house.isnull().sum().sum()

In [ ]:
#Chained Multioutput Regression


models={#'LinearRegression':LinearRegression(),
    'RandomForest':RandomForestRegressor(random_state=42,max_depth=8),
    'GradientBoosting':GradientBoostingRegressor(random_state=42,max_depth=8),
    'AdaBoost':AdaBoostRegressor(random_state=42,base_estimator=DecisionTreeRegressor(random_state=42,max_depth=8)),
    #'Lasso':Lasso(random_state=42),
    #'Ridge':Ridge(random_state=42),
    #'ElasticNet':ElasticNet(random_state=42),
    #'KNeighborsRegressor':KNeighborsRegressor(),
    'DecisionTree':DecisionTreeRegressor(random_state=42,max_depth=8),
    #'LinearSVR':LinearSVR(max_iter=50000,random_state=42),
    #'SVR':  SVR(max_iter=50000)
    }

#num_attributes=['pH_drain_PC' ,'WC_slab1','Rhout' ,                        
#'WC_slab2','RadSum','Pyrgeo','EC_slab1','AbsHumOut','EC_slab2','t_slab1',                       
#'t_slab2','Winddir','HumDef','CO2air','Rhair',                         
#'Tout','EC_drain_PC','Tair','Windsp','co2_dos','Iglob','PARout','Tot_PAR','days','hours' ] 
num_attributes=green_house.columns.drop(['Water_Quantity' ,'Duration_of_Irrigation', 'Irrigation_Time_Intervals' ])

cat_attributes=[]#['Rain']
outputs=['Water_Quantity' ,'Duration_of_Irrigation', 'Irrigation_Time_Intervals']

num_pipline=make_pipeline(SimpleImputer(strategy='mean'),StandardScaler())
cat_pipeline=make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder())

preprocessing=ColumnTransformer([('num', num_pipline, num_attributes),
                                ('cat',cat_pipeline,cat_attributes),
                                ])



from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score,cross_val_predict,cross_validate

outputs=y.columns
train_results=dict()
train_results[outputs[0]]=[] 
train_results[outputs[1]]=[]
train_results[outputs[2]]=[]
test_results=dict()
test_results[outputs[0]]=[] 
test_results[outputs[1]]=[]
test_results[outputs[2]]=[]
conf_interv_results=dict()
conf_interv_results[outputs[0]]=[]
conf_interv_results[outputs[1]]=[]
conf_interv_results[outputs[2]]=[]
train_rmse=dict()
train_r2=dict()
train_mse=dict()
test_rmse=dict()
test_r2=dict()
test_mse=dict()
c1=dict()
c2=dict()

for key in models.keys():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0,shuffle=True,
                                                    stratify=green_house['Rain'])

    full_pipeline = Pipeline([
        ('preprocessing', preprocessing),
        ('models',RegressorChain(models[key],order=[0,1,2],random_state=42)),
    ])
    #train
    score=cross_validate(full_pipeline, X_train, y_train,scoring='neg_root_mean_squared_error', cv=3)
    train_prediction=cross_val_predict(full_pipeline,X_train,y_train,cv=3)
    #train_prediction=min_max_scaler.inverse_transform(train_prediction)
    test_prediction=cross_val_predict(full_pipeline,X_test,y_test,cv=3)
    #train_prediction=min_max_scaler.inverse_transform(test_prediction)

    for i,out in zip(range(0,len(y.columns),1),y.columns):
        #train scores:
        train_rmse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train[out],squared=False)/y[out].mean()
        train_mse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train[out])/y[out].mean()
        train_r2[out]=r2_score(y_true=y_train[out],y_pred=train_prediction[:,i])
        #train result for each output
        train_results[out].append((key,train_rmse[out],train_mse[out],train_r2[out]))
       
        #test scores:
        test_rmse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test[out],squared=False)/y[out].mean()
        test_mse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test[out])/y[out].mean()
        test_r2[out]=r2_score(y_true=y_test[out],y_pred=test_prediction[:,i])
        #test result for each output
        test_results[out].append((key,test_rmse[out],test_mse[out],test_r2[out]))
        
        from scipy import stats
        #confidence_interval:
        confidence = 0.95
        squared_errors = (test_prediction[:,i] - y_test[out]) ** 2
        zscore = stats.norm.ppf((1 + confidence) / 2)
        zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(len(squared_errors))
        c1[out] = np.sqrt(squared_errors.mean() - zmargin)/y[out].mean()
        c2[out]= np.sqrt(squared_errors.mean() + zmargin)/y[out].mean()
        #confidence_intervals for each output:
        conf_interv_results[out].append((key,test_rmse[out],c1[out],c2[out],c2[out]-c1[out]))
        

print(train_results[outputs[0]],'\n',train_results[outputs[1]],'\n',train_results[outputs[2]],'\n')
print(test_results[outputs[0]],'\n',test_results[outputs[1]],'\n',test_results[outputs[2]],'\n')
#print(conf_interv_results[outputs[0]],'\n',conf_interv_results[outputs[1]],'\n',conf_interv_results[outputs[2]],'\n')


In [ ]:
#show training results table:
best_trained_model_idx=dict()
#print('models train Water_Quantity scores:',train_results[outputs[0]])
best_trained_model_idx[outputs[0]]=np.array(train_results[outputs[0]])[:,1].argmin()

#print('models train Duration_of_Irrigation scores:',train_results[outputs[1]])
best_trained_model_idx[outputs[1]]=np.array(train_results[outputs[1]])[:,1].argmin()

#print('models train Water_interv scores:',train_results[outputs[2]])
best_trained_model_idx[outputs[2]]=np.array(train_results[outputs[2]])[:,1].argmin()

#print('models test scores:',test_results)
#best_tested_model_idx=np.array(test_results)[:,1].argmax()


print('best trained model:',train_results[outputs[0]][best_trained_model_idx[outputs[0]]][0],train_results[outputs[0]][best_trained_model_idx[outputs[0]]][1],'\n',
                            train_results[outputs[1]][best_trained_model_idx[outputs[1]]][0],train_results[outputs[1]][best_trained_model_idx[outputs[1]]][1],'\n',
                            train_results[outputs[2]][best_trained_model_idx[outputs[2]]][0],train_results[outputs[2]][best_trained_model_idx[outputs[2]]][1],'\n',
)
train_result=dict()
#print('best tested model:',test_results[best_tested_model_idx][0],test_results[best_tested_model_idx][1])
train_result[outputs[0]]=pd.DataFrame(train_results[outputs[0]],
           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')

train_result[outputs[1]]=pd.DataFrame(train_results[outputs[1]],
           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')

train_result[outputs[2]]=pd.DataFrame(train_results[outputs[2]],
           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
#import dataframe_image as dfi

#dfi.export(train_result, "ModelsTrainingResultsTable.png")
train_result[outputs[0]]


In [ ]:
train_result[outputs[1]]

In [ ]:
train_result[outputs[2]]

In [ ]:
print(conf_interv_results[outputs[0]],'\n',conf_interv_results[outputs[1]],'\n',conf_interv_results[outputs[2]],'\n')

conf_interv_result=dict()
conf_interv_result[outputs[0]]=pd.DataFrame(conf_interv_results[outputs[0]],
           columns=['Model','RMSE','C1','C2','Confidence Interval']).sort_values(by='RMSE',ignore_index=True)

conf_interv_result[outputs[1]]=pd.DataFrame(conf_interv_results[outputs[1]],
           columns=['Model','RMSE','C1','C2','Confidence Interval']).sort_values(by='RMSE',ignore_index=True)

conf_interv_result[outputs[2]]=pd.DataFrame(conf_interv_results[outputs[2]],
           columns=['Model','RMSE','C1','C2','Confidence Interval']).sort_values(by='RMSE',ignore_index=True)

conf_interv_result[outputs[0]]

In [ ]:
conf_interv_result[outputs[1]]

In [ ]:
conf_interv_result[outputs[2]]

In [ ]:
plt.figure(figsize=(6,6))
plt.xticks(range(len(conf_interv_result[outputs[0]])),list(conf_interv_result[outputs[0]]['Model']),
            rotation = 90, ha="right")
plt.title('RMSE Confidence Intevals of Water_Quantity Output\n for Classical ML Models',fontsize=12)
for C1,C2,b in zip(conf_interv_result[outputs[0]]['C1'],conf_interv_result[outputs[0]]['C2'],
                    range(len(conf_interv_result[outputs[0]]))):
    color='#2187bb'
    horizontal_line_width=0.25
    left = b - horizontal_line_width / 4
    right = b + horizontal_line_width / 4
    #plt.plot((b,b),(C1,C2),'ro-',color='orange')
    plt.plot([b, b], [C2, C1], color=color)
    plt.plot([left, right], [C2, C2], color=color)
    plt.plot([left, right], [C1, C1], color=color)
    plt.plot(b, conf_interv_result[outputs[0]]['RMSE'][b], 'o', color='#f44336')
save_fig('RMSE_Confidence_Intevals_of_WQ_for_Classical_ML_Models')

#plt.xticks(range(len(confidence_interval_results)),list(confidence_interval_results['Model']))
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
plt.xticks(range(len(conf_interv_result[outputs[1]])),list(conf_interv_result[outputs[1]]['Model']),
            rotation = 90, ha="right")
plt.title('RMSE Confidence Intevals of Duration of Irrigation\n Output for Classical ML Models',fontsize=12)
for C1,C2,b in zip(conf_interv_result[outputs[1]]['C1'],conf_interv_result[outputs[1]]['C2'],
                    range(len(conf_interv_result[outputs[1]]))):
    color='#2187bb'
    horizontal_line_width=0.25
    left = b - horizontal_line_width / 4
    right = b + horizontal_line_width / 4
    #plt.plot((b,b),(C1,C2),'ro-',color='orange')
    plt.plot([b, b], [C2, C1], color=color)
    plt.plot([left, right], [C2, C2], color=color)
    plt.plot([left, right], [C1, C1], color=color)
    plt.plot(b, conf_interv_result[outputs[1]]['RMSE'][b], 'o', color='#f44336')
save_fig('RMSE_Confidence_Intevals_of_DI_for_Classical_ML_Models')

#plt.xticks(range(len(confidence_interval_results)),list(confidence_interval_results['Model']))
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
plt.xticks(range(len(conf_interv_result[outputs[2]])),list(conf_interv_result[outputs[2]]['Model']),
            rotation = 90, ha="right")
plt.title('RMSE Confidence Intevals of Irrigation time Intevals\n Output for Classical ML Models',fontsize=12)
for C1,C2,b in zip(conf_interv_result[outputs[2]]['C1'],conf_interv_result[outputs[2]]['C2'],
                    range(len(conf_interv_result[outputs[2]]))):
    color='#2187bb'
    horizontal_line_width=0.25
    left = b - horizontal_line_width / 4
    right = b + horizontal_line_width / 4
    #plt.plot((b,b),(C1,C2),'ro-',color='orange')
    plt.plot([b, b], [C2, C1], color=color)
    plt.plot([left, right], [C2, C2], color=color)
    plt.plot([left, right], [C1, C1], color=color)
    plt.plot(b, conf_interv_result[outputs[2]]['RMSE'][b], 'o', color='#f44336')
save_fig('RMSE_Confidence_Intevals_of_ITI_for_Classical_ML_Models')

#plt.xticks(range(len(confidence_interval_results)),list(confidence_interval_results['Model']))
plt.show()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error,r2_score
from scipy.stats import randint
from sklearn.multioutput import RegressorChain



outputs=y.columns
train_results=dict()
train_results[outputs[0]]=[] 
train_results[outputs[1]]=[]
train_results[outputs[2]]=[]
test_results=dict()
test_results[outputs[0]]=[] 
test_results[outputs[1]]=[]
test_results[outputs[2]]=[]
conf_interv_results=dict()
conf_interv_results[outputs[0]]=[]
conf_interv_results[outputs[1]]=[]
conf_interv_results[outputs[2]]=[]
train_rmse=dict()
train_r2=dict()
train_mse=dict()
test_rmse=dict()
test_r2=dict()
test_mse=dict()
c1=dict()
c2=dict()



estimator=RandomForestRegressor(n_estimators= 100,criterion='squared_error',
    max_depth= None,min_samples_split= 2,min_samples_leaf= 1,min_weight_fraction_leaf= 0,max_features='sqrt',
    max_leaf_nodes= None,min_impurity_decrease= 0,bootstrap= True,oob_score=False,n_jobs= -1,
    ccp_alpha=0,max_samples= None,random_state=42)

best_models=dict()

best_models['Chain_RF']=RegressorChain(base_estimator=estimator,order=[0,1,2],random_state=42)
full_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('Chain_RF',best_models['Chain_RF'] )
                        ])
       
param_grid=[{ 
                'Chain_RF__base_estimator__max_depth':[8],
                'Chain_RF__base_estimator__n_estimators':[300],
                'Chain_RF__base_estimator__criterion':['squared_error'],
                'Chain_RF__base_estimator__min_samples_split':[2],
                'Chain_RF__base_estimator__max_samples':[None],
                'Chain_RF__base_estimator__max_features':['sqrt'],
                'Chain_RF__base_estimator__ccp_alpha':[0],              
            }]
grid_search=GridSearchCV(full_pipeline,param_grid,cv=10,scoring='neg_root_mean_squared_error',verbose=1)


In [ ]:
grid_search.fit(X_train,y_train)


In [ ]:
train_prediction=cross_val_predict(grid_search.best_estimator_,X_train,y_train,cv=10)
test_prediction=cross_val_predict(grid_search.best_estimator_,X_test,y_test,cv=10)

#rmse_percentage=rmse.mean()/y.mean()
#print('rmse percentage:',rmse_percentage)
#print(grid_search.best_params_)

In [ ]:


for i,out in zip(range(0,len(y.columns),1),y.columns):
    #train scores:
    train_rmse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train[out],squared=False)/y[out].mean()
    train_mse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train[out])/y[out].mean()
    train_r2[out]=r2_score(y_true=y_train[out],y_pred=train_prediction[:,i])
    #train result for each output
    train_results[out].append(('Chain_RF',train_rmse[out],train_mse[out],train_r2[out]))
   
    #test scores:
    test_rmse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test[out],squared=False)/y[out].mean()
    test_mse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test[out])/y[out].mean()
    test_r2[out]=r2_score(y_true=y_test[out],y_pred=test_prediction[:,i])
    #test result for each output
    test_results[out].append(('Chain_RF',test_rmse[out],test_mse[out],test_r2[out]))
    
    # Plot predicted vs actual
    plt.style.use("ggplot")
    plt.scatter(y_test[out], test_prediction[:,i],color='steelblue')
    plt.xlabel(f'Actual {out}')
    plt.ylabel(f'Predicted {out}')
    plt.title(f'predicted vs actual {out} \nplot for Chain RF',fontsize=12)
    # overlay the regression line
    z = np.polyfit(y_test[out], test_prediction[:,i], 1)
    p = np.poly1d(z)
    plt.plot(y_test[out],p(y_test[out]),color='red')
    #plt.grid()
    save_fig(f'predicted vs actual {out} plot for Chain RF')
    plt.show()



    from scipy import stats
    #confidence_interval:
    confidence = 0.95
    squared_errors = (test_prediction[:,i] - y_test[out]) ** 2
    zscore = stats.norm.ppf((1 + confidence) / 2)
    zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(len(squared_errors))
    c1[out] = np.sqrt(squared_errors.mean() - zmargin)/y[out].mean()
    c2[out] = np.sqrt(squared_errors.mean() + zmargin)/y[out].mean()
    #confidence_intervals for each output:
    conf_interv_results[out].append(('Chain_RF',test_rmse[out],c1[out],c2[out],c2[out]-c1[out]))
    
print(train_results[outputs[0]],'\n',train_results[outputs[1]],'\n',train_results[outputs[2]],'\n')
print(test_results[outputs[0]],'\n',test_results[outputs[1]],'\n',test_results[outputs[2]],'\n')
print(conf_interv_results[outputs[0]],'\n',conf_interv_results[outputs[1]],'\n',conf_interv_results[outputs[2]],'\n')


In [ ]:
y_test[out].values

In [ ]:
for pred_out,i in zip(outputs,range(len(outputs))):
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(test_prediction[:,i][15:50],label='predected', marker=".")
    plt.plot(y_test[pred_out].values[15:50],label='Actual',marker=".")

    plt.title(f"Predicted Vs. Actual of {pred_out} \nsamples of Chain RF",fontsize=12)
    plt.xlabel("samples")
    plt.ylabel(f"{pred_out}")
    plt.legend(loc="lower left")
    save_fig(f"Predicted Vs. Actual of {pred_out} in discrete time series points of Chain RF")

    plt.show()

In [ ]:
#grid_search_test_rmse=dict()
#for i,out in zip(range(0,len(y.columns),1),y.columns):
#
#    grid_search_test_rmse[out]=mean_squared_error(y_pred=pred[:,i],y_true=y_train[out],squared=False)/y[out].mean()
#print(grid_search_test_rmse)

In [ ]:
#print('best score=',-grid_search.best_score_/y.mean())
#rmse=-cross_val_score(full_pipeline,X_train,y_train,scoring='neg_root_mean_squared_error',cv=10)
#rmse.mean()
#rmse_percentage=rmse.mean()/y.mean()
#print('rmse percentage:',rmse_percentage)
#print(grid_search.best_params_)

In [ ]:
best_params=grid_search.best_params_
#best_params['RandomForestRegressor__base_estimator']=grid_search.best_estimator_['RandomForestRegressor__base_estimator'].n_estimators_
GBR_best_param=pd.DataFrame.from_dict(grid_search.best_params_,orient='index',columns=['best value'])
GBR_best_param['best value']

In [ ]:
feature_importances=dict()
features_imoprtance=dict()
for out,estimator in zip(outputs,grid_search.best_estimator_['Chain_RF'].estimators_):
    feature_importances[out]=estimator.feature_importances_.round(2)
    features_imoprtance[out]=sorted(zip(feature_importances[out],
           grid_search.best_estimator_["preprocessing"].get_feature_names_out()),reverse=True)

    features_imoprtance[out]=pd.DataFrame(features_imoprtance[out],
               columns=['importance','features'])

    #import dataframe_image as dfi
    #
    #dfi.export(features_imoprtance, "FeatureImportanceTable.png")
#features_imoprtance[outputs[0]]



In [ ]:
#features_imoprtance[outputs[1]]


In [ ]:
#features_imoprtance[outputs[2]]


In [ ]:
#feature_importances

In [ ]:
for i,out in zip(range(0,len(outputs),1),outputs):
    #features_imoprtance[out].set_index('features')
    features_imoprtance[out].index=features_imoprtance[out]['features']
    #plt.subplot(1,3,i+1)

    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    (features_imoprtance[out]['importance']).plot.bar(ax=ax,color='steelblue')

    #(features_imoprtance[out]['importance']/features_imoprtance[out]['importance'].max()*100).plot.bar(ax=ax)
    ax.set_title(f'{out} Features Importance',fontsize=12)
    ax.set_xlabel('Features')
    ax.set_ylabel("Features Importance")
    save_fig(f'Features Importance of {out} Output')

    plt.show()

In [ ]:
#Using ANN for a multioutput regresion task:
import tensorflow as tf
from tensorflow import keras 
from keras import layers
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
y=pd.DataFrame(green_house.loc[:,['Water_Quantity','Duration_of_Irrigation','Irrigation_Time_Intervals']])

X=green_house.drop(['Duration_of_Irrigation','Water_Quantity','Irrigation_Time_Intervals'],axis=1).copy()
print(y.shape)
print(X.shape)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0,shuffle=True,
                                                    stratify=green_house['Rain'])





X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,test_size=0.10, random_state=42)

X_train=preprocessing.fit_transform(X_train)
X_test=preprocessing.transform(X_test)
X_valid=preprocessing.transform(X_valid)

print('X_train',X_train.shape)
print('X_test',X_test.shape)
print('X_valid',X_valid.shape)
from sklearn.preprocessing import MinMaxScaler
min_max_scaler=MinMaxScaler()

y_train=min_max_scaler.fit_transform(y_train)
y_test=min_max_scaler.transform(y_test)
y_valid=min_max_scaler.transform(y_valid)
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('y_valid',y_valid.shape)


y_train_pred=[]
print(X_train.shape[1:][0])

norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
leaky_relu = keras.layers.LeakyReLU(alpha=0.3)  # defaults to alpha=0.3
best_models['Parallel_Out_ANN'] = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(500, activation='swish',kernel_initializer="he_normal"),
    tf.keras.layers.Dense(150, activation='swish',kernel_initializer="he_normal"),
    tf.keras.layers.Dense(500, activation='swish',kernel_initializer="he_normal"),
    tf.keras.layers.Dense(150, activation='swish',kernel_initializer="he_normal"),
    tf.keras.layers.Dense(500, activation='swish',kernel_initializer="he_normal"),
    tf.keras.layers.Dense(150, activation='swish',kernel_initializer="he_normal"),

    tf.keras.layers.Dense(3,activation='linear',kernel_initializer="he_normal")
])
checkpoint_path = "training_1/par_cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                   save_weights_only=True)


early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=30,
                                                     restore_best_weights=True)
best_models['Parallel_Out_ANN'].compile(loss="huber", optimizer=optimizer, metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)
history = best_models['Parallel_Out_ANN'].fit(X_train, y_train, epochs=500,
                    validation_data=(X_valid, y_valid),batch_size=64,callbacks=[checkpoint_cb, early_stopping_cb])
#mse_test, rmse_test = model.evaluate(X_test, y_test)
#X_new = X_test[:3]
# plot loss during training
plt.title('Train Vs.Valid Loss of Parallel_Out_ANN',fontsize=12)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='valid')
plt.legend()
save_fig('Train Vs.Valid Loss of Parallel_Out_ANN')

plt.show()



In [ ]:
len(history.history['loss'])

In [ ]:
train_RMSE = best_models['Parallel_Out_ANN'].evaluate(X_train, y_train,return_dict=True, verbose=1)
val_RMSE = best_models['Parallel_Out_ANN'].evaluate(X_valid, y_valid,return_dict=True, verbose=1)

test_RMSE = best_models['Parallel_Out_ANN'].evaluate(X_test, y_test,return_dict=True, verbose=1)
print(f'Train: {train_RMSE},\n val: {val_RMSE},\nTest: {test_RMSE}')

In [ ]:
train_prediction = best_models['Parallel_Out_ANN'].predict((X_train))
train_prediction.reshape(len(y_train),3)

test_prediction = best_models['Parallel_Out_ANN'].predict((X_test))
test_prediction.reshape(len(y_test),3)

test_prediction


In [ ]:
train_prediction=min_max_scaler.inverse_transform(train_prediction)
test_prediction=min_max_scaler.inverse_transform(test_prediction)

In [ ]:
y_test

In [ ]:
test_prediction

In [ ]:

y_train_inv=min_max_scaler.inverse_transform(y_train)
y_test_inv=min_max_scaler.inverse_transform(y_test)


In [ ]:
y_train_inv.shape

In [ ]:
test_prediction.shape

In [ ]:

for i,out in zip(range(0,len(y.columns),1),y.columns):
    #train scores:
    train_rmse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train_inv[:,i],squared=False)/y[out].mean()
    train_mse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train_inv[:,i])/y[out].mean()
    train_r2[out]=r2_score(y_true=y_train_inv[:,i],y_pred=train_prediction[:,i])
    #train result for each output
    train_results[out].append(('Parallel_Out_ANN',train_rmse[out],train_mse[out],train_r2[out]))
   
    #test scores:
    test_rmse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test_inv[:,i],squared=False)/y[out].mean()
    test_mse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test_inv[:,i])/y[out].mean()
    test_r2[out]=r2_score(y_true=y_test_inv[:,i],y_pred=test_prediction[:,i])
    #test result for each output
    test_results[out].append(('Parallel_Out_ANN',test_rmse[out],test_mse[out],test_r2[out]))
    
    # Plot predicted vs actual
    plt.style.use("ggplot")
    plt.scatter(y_test_inv[:,i], test_prediction[:,i],color='steelblue')
    plt.xlabel(f'Actual {out}')
    plt.ylabel(f'Predicted {out}')
    plt.title(f'predicted vs actual {out} plot \nfor Parallel_Out_ANN',fontsize=12)
    # overlay the regression line
    z = np.polyfit(y_test_inv[:,i], test_prediction[:,i], 1)
    p = np.poly1d(z)
    plt.plot(y_test_inv[:,i],p(y_test_inv[:,i]),color='red')
    #plt.grid()
    save_fig(f'predicted vs actual {out} plot for Parallel_Out_ANN')
    plt.show()

    from scipy import stats
    #confidence_interval:
    confidence = 0.95
    squared_errors = (test_prediction[:,i] - y_test_inv[:,i]) ** 2
    zscore = stats.norm.ppf((1 + confidence) / 2)
    zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(len(squared_errors))
    c1[out],c2[out] = np.sqrt(squared_errors.mean() - zmargin)/y[out].mean(), np.sqrt(squared_errors.mean() 
                    + zmargin)/y[out].mean()
    #confidence_intervals for each output:
    conf_interv_results[out].append(('Parallel_Out_ANN',test_rmse[out],c1[out],c2[out],c2[out]-c1[out]))
        

print(train_results[outputs[0]],'\n',train_results[outputs[1]],'\n',train_results[outputs[2]],'\n')
print(test_results[outputs[0]],'\n',test_results[outputs[1]],'\n',test_results[outputs[2]],'\n')
print(conf_interv_results[outputs[0]],'\n',conf_interv_results[outputs[1]],'\n',conf_interv_results[outputs[2]],'\n')


In [ ]:
for pred_out,i in zip(outputs,range(len(outputs))):
    plt.style.use("ggplot")
    predictions=test_prediction[:,i]
    plt.figure()
    plt.plot(predictions[15:50],label='predected', marker=".")
    plt.plot(y_test_inv[:,i][15:50],label='Actual',marker=".")

    plt.title(f"Predicted Vs. Actual of {pred_out} \nsamples of Parallel_Out_ANN",fontsize=12)
    plt.xlabel("samples")
    plt.ylabel(f"{pred_out}")
    plt.legend(loc="lower left")
    save_fig(f"Predicted Vs. Actual of {pred_out} in discrete time series points of Parallel_Out_ANN")

    plt.show()

In [ ]:
y_train_pred=best_models['Parallel_Out_ANN'].predict(X_train)

In [ ]:
y_train_pred

In [ ]:
#model=model.load_weights("my_checkpoints")
model_rms_train=mean_squared_error(y_pred=y_train_pred[:,0],y_true=y_train[:,0],squared=False)#/y_train[outputs[0]].mean()
model_rms_train

In [ ]:
model_rms_train=mean_squared_error(y_pred=y_train_pred[:,1],y_true=y_train[:,1],squared=False)#/y[outputs[1]].mean()
model_rms_train

In [ ]:
model_rms_train=mean_squared_error(y_pred=y_train_pred[:,2],y_true=y_train[:,2],squared=False)#/y[outputs[2]].mean()
model_rms_train

In [ ]:
y_pred_test = best_models['Parallel_Out_ANN'].predict(X_test)

In [ ]:
model_rms_test=mean_squared_error(y_pred=y_pred_test[:,0],y_true=y_test[:,0],squared=False)#/y_train[outputs[0]].mean()
model_rms_test

In [ ]:
model_rms_test=mean_squared_error(y_pred=y_pred_test[:,1],y_true=y_test[:,1],squared=False)#/y_train[outputs[1]].mean()
model_rms_test

In [ ]:
model_rms_train=mean_squared_error(y_pred=y_pred_test[:,2],y_true=y_test[:,2],squared=False)#/y_train[outputs[1]].mean()
model_rms_train

In [ ]:
from keras import layers
#tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0,shuffle=True,
                                                    stratify=green_house['Rain'])





X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,test_size=0.10, random_state=42)

X_train=preprocessing.fit_transform(X_train)
X_test=preprocessing.transform(X_test)
X_valid=preprocessing.transform(X_valid)

print('X_train',X_train.shape)
print('X_test',X_test.shape)
print('X_valid',X_valid.shape)
from sklearn.preprocessing import MinMaxScaler
min_max_scaler=MinMaxScaler()

y_train=min_max_scaler.fit_transform(y_train)
y_test=min_max_scaler.transform(y_test)
y_valid=min_max_scaler.transform(y_valid)
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('y_valid',y_valid.shape)


y_train_pred=[]
print(X_train.shape[1:][0])

input_wide = layers.Input(shape=X_train.shape[1:])  # features 0 to 4
#input_deep = tf.keras.layers.Input(shape=[6])  # features 2 to 7
norm_layer_wide = layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
#norm_deep = norm_layer_deep(input_deep)

#hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = layers.Dense(500, activation='swish',kernel_initializer="he_normal")(norm_wide)
hidden3 = layers.Dense(150, activation='swish',kernel_initializer="he_normal")(hidden2)

hidden4 = layers.Dense(500, activation='swish',kernel_initializer="he_normal")(norm_wide)
hidden5 = layers.Dense(150, activation='swish',kernel_initializer="he_normal")(hidden4)

hidden6 = layers.Dense(500, activation='swish',kernel_initializer="he_normal")(norm_wide)
hidden7 = layers.Dense(150, activation='swish',kernel_initializer="he_normal")(hidden6)

output_0 = layers.Dense(1,activation='linear')(hidden3)
norm_out_0=layers.BatchNormalization()(output_0)
concat_0 = tf.keras.layers.concatenate([hidden5, norm_out_0])
output_1 = tf.keras.layers.Dense(1,activation='linear')(concat_0)
norm_out_1=layers.BatchNormalization()(output_1)

concat_1 = tf.keras.layers.concatenate([hidden7, norm_out_0,norm_out_1])
output_2 = tf.keras.layers.Dense(1,activation='linear')(concat_1)


best_models['Cascaded_Out_ANN'] = tf.keras.Model(inputs=[input_wide],
                       outputs=[output_0, output_1,output_2])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
checkpoint_path = "training_1/cas_cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                   save_best_model=True)


early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=30,
                                                     restore_best_weights=True)


best_models['Cascaded_Out_ANN'].compile(loss=("huber", "huber","huber"), loss_weights=(1,1, 1), optimizer=optimizer,
              metrics=["RootMeanSquaredError"])


norm_layer_wide.adapt(X_train)
history = best_models['Cascaded_Out_ANN'].fit(
    (X_train), (y_train[:,0], y_train[:,1],y_train[:,2]), epochs=300,
    validation_data=((X_valid), (y_valid[:,0], y_valid[:,1],y_valid[:,2])),
    batch_size=128,callbacks=[checkpoint_cb, early_stopping_cb]
)

plt.title('Train Vs.Valid Loss of Cascaded_Out_ANN ',fontsize=12)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='valid')
save_fig('Train Vs.Valid Loss of Cascaded_Out_ANN ')
plt.legend()
plt.show()

In [ ]:
y_train

In [ ]:
#os.listdir(checkpoint_dir)
##best_models['Cascaded_Out_ANN']=best_models['Cascaded_Out_ANN'].load_weights(checkpoint_path)
#best_models['Cascaded_Out_ANN']=keras.models.load_model(checkpoint_path)

In [ ]:
train_RMSE = best_models['Cascaded_Out_ANN'].evaluate((X_train), (y_train[:,0],y_train[:,1],y_train[:,2]),return_dict=True, verbose=1)
val_RMSE = best_models['Cascaded_Out_ANN'].evaluate((X_valid), (y_valid[:,0],y_valid[:,1],y_valid[:,2]),return_dict=True, verbose=1)

test_RMSE = best_models['Cascaded_Out_ANN'].evaluate((X_test), (y_test[:,0],y_test[:,1],y_test[:,2]),return_dict=True, verbose=1)
print(f'Train: {train_RMSE},\n val: {val_RMSE},\n Test: {test_RMSE}')

In [ ]:
test_RMSE

In [ ]:
train_out1,train_out2,train_out3 = best_models['Cascaded_Out_ANN'].predict((X_train))
train_prediction=np.c_[train_out1,train_out2,train_out3]
train_prediction.shape

In [ ]:
test_out1,test_out2,test_out3 = best_models['Cascaded_Out_ANN'].predict((X_test))
test_prediction=np.c_[test_out1,test_out2,test_out3]
test_prediction.shape

In [ ]:

#train_out1,train_out2,train_out3 = best_models['Cascaded_Out_ANN'].predict((X_train))
##train_prediction=np.asarray(train_prediction)[:,:,0].reshape(len(y_train),3)
#train_prediction=pd.concat([train_out1,train_out2,train_out3],axis=1)
##train_prediction[0]=train_prediction[0].reshape(1,-1)
##train_prediction[1]=train_prediction[1].reshape(1,-1)
##train_prediction[2]=train_prediction[2].reshape(1,-1)
##train_prediction=np.asarray(train_prediction)[:,:,0]
#test_prediction = best_models['Cascaded_Out_ANN'].predict((X_test))
#test_prediction=np.asarray(test_prediction)[:,:,0].reshape(len(y_test),3)
#
##test_prediction[0]=test_prediction[0].reshape(1,-1)
##test_prediction[1]=test_prediction[1].reshape(1,-1)
##test_prediction[2]=test_prediction[2].reshape(1,-1)
##test_prediction=np.asarray(test_prediction)[:,:,0]
test_prediction




In [ ]:
y_test

In [ ]:
#test_prediction=np.array(test_prediction)[:,:,0]
#test_prediction.shape

In [ ]:
#test_prediction[0,:]

In [ ]:
#test_prediction=test_prediction.reshape(-1,3)
#test_prediction.shape

In [ ]:
#train_prediction=np.array(train_prediction)[:,:,0].reshape(-1,3)

In [ ]:
#test_prediction=np.c_[test_prediction[0],test_prediction[1],test_prediction[2]]
#train_prediction=np.c_[train_prediction[0],train_prediction[1],train_prediction[2]]

In [ ]:
y_test

In [ ]:
train_prediction.shape

In [ ]:
train_prediction=min_max_scaler.inverse_transform(train_prediction)
test_prediction=min_max_scaler.inverse_transform(test_prediction)

In [ ]:
y_train_inv=min_max_scaler.inverse_transform(y_train)
y_test_inv=min_max_scaler.inverse_transform(y_test)

In [ ]:
test_prediction

In [ ]:
y_test_inv

In [ ]:
#best_models['Cascaded_Out_ANN']=keras.models.load_model("my_checkpoints_best")
#train_prediction = best_models['Cascaded_Out_ANN'].predict((X_train))
#test_prediction = best_models['Cascaded_Out_ANN'].predict((X_test))
#train_prediction = np.array(train_prediction).reshape(-1,3)
#test_prediction=np.array(test_prediction).reshape(-1,3)
for i,out in zip(range(0,len(y.columns),1),y.columns):
    #train scores:
    train_rmse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train_inv[:,i],squared=False)/y[out].mean()
    train_mse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=y_train_inv[:,i])/y[out].mean()
    train_r2[out]=r2_score(y_true=y_train_inv[:,i],y_pred=train_prediction[:,i])
    #train result for each output
    train_results[out].append(('Cascaded_Out_ANN',train_rmse[out],train_mse[out],train_r2[out]))
   
    #test scores:
    test_rmse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test_inv[:,i],squared=False)/y[out].mean()
    test_mse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=y_test_inv[:,i])/y[out].mean()
    test_r2[out]=r2_score(y_true=y_test_inv[:,i],y_pred=test_prediction[:,i])
    #test result for each output
    test_results[out].append(('Cascaded_Out_ANN',test_rmse[out],test_mse[out],test_r2[out]))
    
    # Plot predicted vs actual
    plt.style.use("ggplot")
    plt.scatter(y_test_inv[:,i], test_prediction[:,i],color='steelblue')
    plt.xlabel(f'Actual {out}')
    plt.ylabel(f'Predicted {out}')
    plt.title(f'predicted vs actual {out} plot\n for Cascaded_Out_ANN',fontsize=12)
    # overlay the regression line
    z = np.polyfit(y_test_inv[:,i], test_prediction[:,i], 1)
    p = np.poly1d(z)
    plt.plot(y_test_inv[:,i],p(y_test_inv[:,i]),color='red')
    #plt.grid()
    save_fig(f'predicted vs actual {out} plot for Cascaded_Out_ANN')
    plt.show()

    from scipy import stats
    #confidence_interval:
    confidence = 0.95
    squared_errors = (test_prediction[:,i] - y_test_inv[:,i]) ** 2
    zscore = stats.norm.ppf((1 + confidence) / 2)
    zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(len(squared_errors))
    c1[out],c2[out] = np.sqrt(squared_errors.mean() - zmargin)/y[out].mean(), np.sqrt(squared_errors.mean() 
                    + zmargin)/y[out].mean()
    #confidence_intervals for each output:
    conf_interv_results[out].append(('Cascaded_Out_ANN',test_rmse[out],c1[out],c2[out],c2[out]-c1[out]))
        

#print(train_results[outputs[0]],'\n',train_results[outputs[1]],'\n',train_results[outputs[2]],'\n')
print(test_results[outputs[0]],'\n',test_results[outputs[1]],'\n',test_results[outputs[2]],'\n')
#print(conf_interv_results[outputs[0]],'\n',conf_interv_results[outputs[1]],'\n',conf_interv_results[outputs[2]],'\n')


In [ ]:
#np.array(test_prediction)[0,:].reshape(1,-1)[0,:]

In [ ]:
y_test[:,0]

In [ ]:
eval_results = best_models['Cascaded_Out_ANN'].evaluate((X_test), (y_test[:,0], y_test[:,1], y_test[:,2]))
#weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results
eval_results

In [ ]:
y_pred_0, y_pred_1,y_pred_2 = best_models['Cascaded_Out_ANN'].predict((X_train))

In [ ]:
model_rms_train=mean_squared_error(y_pred=y_pred_0,y_true=y_train[:,0],squared=False)#/y_train[outputs[0]].mean()
model_rms_train

In [ ]:
model_rms_train=mean_squared_error(y_pred=y_pred_1,y_true=y_train[:,1],squared=False)#/y_train[outputs[1]].mean()
model_rms_train

In [ ]:
model_rms_train=mean_squared_error(y_pred=y_pred_2,y_true=y_train[:,2],squared=False)#/y_train[outputs[1]].mean()
model_rms_train

In [ ]:
#model=model.load_weights("my_checkpoints")
#model.evaluate(X_test, y_test)

In [ ]:
y_pred_0, y_pred_1,y_pred_2 = best_models['Cascaded_Out_ANN'].predict((X_test))

In [ ]:
model_rms_test=mean_squared_error(y_pred=y_pred_0,y_true=y_test[:,0],squared=False)#/y_train[outputs[0]].mean()
model_rms_test

In [ ]:
model_rms_test=mean_squared_error(y_pred=y_pred_1,y_true=y_test[:,1],squared=False)#/y_train[outputs[1]].mean()
model_rms_test

In [ ]:
model_rms_test=mean_squared_error(y_pred=y_pred_2,y_true=y_test[:,2],squared=False)#/y_train[outputs[1]].mean()
model_rms_test

In [ ]:
#show training results table:
best_trained_model_idx=dict()
for out in outputs:
    print('best models train {out} scores:',train_results[outputs[out]])
    best_trained_model_idx[outputs[out]]=np.array(train_results[outputs[out]])[:,1].argmin()


    #print('best models test scores:',test_results)
    #best_tested_model_idx=np.array(test_results)[:,1].argmax()

print('best trained model:',train_results[outputs[out]][best_trained_model_idx[outputs[out]]][0],
                            train_results[outputs[out]][best_trained_model_idx[outputs[out]]][1],'\n')
train_result=dict()
#print('best tested model:',test_results[best_tested_model_idx][0],test_results[best_tested_model_idx][1])
train_result[outputs[0]]=pd.DataFrame(train_results[outputs[0]],
           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')

train_result[outputs[1]]=pd.DataFrame(train_results[outputs[1]],
           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')

train_result[outputs[2]]=pd.DataFrame(train_results[outputs[2]],
           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
#import dataframe_image as dfi
#
#dfi.export(train_result[outputs[0]], f'ModelsTrainingResults{outputs[0]}Table.png')
#dfi.export(train_result[outputs[1]], f"ModelsTrainingResults{outputs[1]}Table.png")
#dfi.export(train_result[outputs[2]], f"ModelsTrainingResults{outputs[2]}Table.png")

train_result[outputs[0]]


In [ ]:
train_result[outputs[1]]


In [ ]:
train_result[outputs[2]]


In [ ]:
#show testing results table:
best_tested_model_idx=dict()
print('best models test Water_Quantity scores:',test_results[outputs[0]])
best_tested_model_idx[outputs[0]]=np.array(test_results[outputs[0]])[:,1].argmin()

print('best models test Duration_of_Irrigation scores:',test_results[outputs[1]])
best_tested_model_idx[outputs[1]]=np.array(test_results[outputs[1]])[:,1].argmin()

print('best models test Water_interv scores:',test_results[outputs[2]])
best_tested_model_idx[outputs[2]]=np.array(test_results[outputs[2]])[:,1].argmin()

#print('best models test scores:',test_results)
#best_tested_model_idx=np.array(test_results)[:,1].argmax()


print('best tested model:',test_results[outputs[0]][best_tested_model_idx[outputs[0]]][0],test_results[outputs[0]][best_tested_model_idx[outputs[0]]][1],'\n',
                            test_results[outputs[1]][best_tested_model_idx[outputs[1]]][0],test_results[outputs[1]][best_tested_model_idx[outputs[1]]][1],'\n',
                            test_results[outputs[2]][best_tested_model_idx[outputs[2]]][0],test_results[outputs[2]][best_tested_model_idx[outputs[2]]][1],'\n',
)
test_result=dict()
#print('best tested model:',test_results[best_tested_model_idx][0],test_results[best_tested_model_idx][1])
test_result[outputs[0]]=pd.DataFrame(test_results[outputs[0]],
           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')

test_result[outputs[1]]=pd.DataFrame(test_results[outputs[1]],
           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')

test_result[outputs[2]]=pd.DataFrame(test_results[outputs[2]],
           columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
#import dataframe_image as dfi

#dfi.export(test_result, "ModelstestingResultsTable.png")
test_result[outputs[0]]


In [ ]:
test_result[outputs[1]]


In [ ]:
test_result[outputs[2]]


In [ ]:
print(conf_interv_results[outputs[0]],'\n',conf_interv_results[outputs[1]],'\n',conf_interv_results[outputs[2]],'\n')

conf_interv_result=dict()
conf_interv_result[outputs[0]]=pd.DataFrame(conf_interv_results[outputs[0]],
           columns=['Model','RMSE','C1','C2','Confidence Interval']).sort_values(by='RMSE',ignore_index=True)

conf_interv_result[outputs[1]]=pd.DataFrame(conf_interv_results[outputs[1]],
           columns=['Model','RMSE','C1','C2','Confidence Interval']).sort_values(by='RMSE',ignore_index=True)

conf_interv_result[outputs[2]]=pd.DataFrame(conf_interv_results[outputs[2]],
           columns=['Model','RMSE','C1','C2','Confidence Interval']).sort_values(by='RMSE',ignore_index=True)

conf_interv_result[outputs[0]]

In [ ]:
conf_interv_result[outputs[1]]

In [ ]:
conf_interv_result[outputs[2]]

In [ ]:
#conf_interv_result[outputs[0]].drop('KNeighborsRegressor')
plt.figure(figsize=(6,6))
plt.xticks(range(len(conf_interv_result[outputs[0]])),list(conf_interv_result[outputs[0]]['Model']),
            rotation = 90, ha="right")
plt.title('RMSE Confidence Intervals Comparison of\n Water_Quantity Output',fontsize=12)
for C1,C2,b in zip(conf_interv_result[outputs[0]]['C1'],conf_interv_result[outputs[0]]['C2'],
                    range(len(conf_interv_result[outputs[0]]))):
    color='#2187bb'
    horizontal_line_width=0.25
    left = b - horizontal_line_width / 4
    right = b + horizontal_line_width / 4
    #plt.plot((b,b),(C1,C2),'ro-',color='orange')
    plt.plot([b, b], [C2, C1], color=color)
    plt.plot([left, right], [C2, C2], color=color)
    plt.plot([left, right], [C1, C1], color=color)
    plt.plot(b, conf_interv_result[outputs[0]]['RMSE'][b], 'o', color='#f44336')

save_fig(f'RMSE Confidence Intervals of Water_Quantity Output')
#plt.xticks(range(len(confidence_interval_results)),list(confidence_interval_results['Model']))
plt.show()

In [ ]:
#conf_interv_result[outputs[1]].drop('KNeighborsRegressor')

plt.figure(figsize=(6,6))
plt.xticks(range(len(conf_interv_result[outputs[1]])),list(conf_interv_result[outputs[1]]['Model']),
            rotation = 90, ha="right")
plt.title('RMSE Confidence Intervals Comparison of \nDuration_of_Irrigation Output',fontsize=12)
for C1,C2,b in zip(conf_interv_result[outputs[1]]['C1'],conf_interv_result[outputs[1]]['C2'],
                    range(len(conf_interv_result[outputs[1]]))):
    color='#2187bb'
    horizontal_line_width=0.25
    left = b - horizontal_line_width / 4
    right = b + horizontal_line_width / 4
    #plt.plot((b,b),(C1,C2),'ro-',color='orange')
    plt.plot([b, b], [C2, C1], color=color)
    plt.plot([left, right], [C2, C2], color=color)
    plt.plot([left, right], [C1, C1], color=color)
    plt.plot(b, conf_interv_result[outputs[1]]['RMSE'][b], 'o', color='#f44336')

save_fig(f'RMSE Confidence Intervals of Duration_of_Irrigation Output')
#plt.xticks(range(len(confidence_interval_results)),list(confidence_interval_results['Model']))
plt.show()

In [ ]:
#conf_interv_result[outputs[2]].drop('KNeighborsRegressor')
plt.figure(figsize=(6,6))
plt.xticks(range(len(conf_interv_result[outputs[2]])),list(conf_interv_result[outputs[2]]['Model']),
            rotation = 90, ha="right")
plt.title('RMSE Confidence Intervals Comparison of \nIrrigation_Time_Intervals Output',fontsize=12)
for C1,C2,b in zip(conf_interv_result[outputs[2]]['C1'],conf_interv_result[outputs[2]]['C2'],
                    range(len(conf_interv_result[outputs[2]]))):
    color='#2187bb'
    horizontal_line_width=0.25
    left = b - horizontal_line_width / 4
    right = b + horizontal_line_width / 4
    #plt.plot((b,b),(C1,C2),'ro-',color='orange')
    plt.plot([b, b], [C2, C1], color=color)
    plt.plot([left, right], [C2, C2], color=color)
    plt.plot([left, right], [C1, C1], color=color)
    plt.plot(b, conf_interv_result[outputs[2]]['RMSE'][b], 'o', color='#f44336')

save_fig(f'RMSE Confidence Intervals of Irrigation_Time_Intervals Output')

#plt.xticks(range(len(confidence_interval_results)),list(confidence_interval_results['Model']))
plt.show()

In [ ]:
test_prediction

In [ ]:

predictions=np.array(test_prediction)[2,:].reshape(1,-1)[0,:]
predictions.shape

In [ ]:
y_test[:,2]

In [ ]:
outputs
#range(len(outputs)-1)

In [ ]:
for pred_out,i in zip(outputs,range(len(outputs))):
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(test_prediction[:,i][15:50],label='predected', marker=".")
    plt.plot(y_test_inv[:,i][15:50],label='Actual',marker=".")

    plt.title(f"Predicted Vs. Actual of {pred_out} \nsamples of Cascaded_Out_ANN",fontsize=12)
    plt.xlabel("samples")
    plt.ylabel(f"{pred_out}")
    plt.legend(loc="lower left")
    save_fig(f"Predicted Vs. Actual of {pred_out} in discrete time series of Cascaded_Out_ANN")

    plt.show()

In [ ]:
from keras.utils.vis_utils import plot_model,model_to_dot
from graphviz import Source
import pydot

#model_to_dot(best_models['Cascaded_Out_ANN'])
plot_model(best_models['Cascaded_Out_ANN'], to_file='Cascaded_Out_ANN_Archeticture.png', show_shapes=True, show_layer_names=True)
#
#
#Source.from_file("Cascaded_Out_ANN_Archeticture.dot")

In [ ]:
#keras.models.save_model(models['Cascaded_Out_ANN'],'Cascaded_Out_ANN.hdf')

In [ ]:
#tf.keras.utils.plot_model(
#models['Cascaded_Out_ANN'],
#to_file=r"C:\Users\user\Desktop\iman\Master\RoboticsAndControlSystems\Project\Code\IrrigationControlSystem\model.png",
#show_shapes=True,
#show_dtype=False,
#show_layer_names=True,
#rankdir="TB",
#expand_nested=True,
#dpi=96,
#layer_range=None,
#show_layer_activations=True,
#)

In [ ]:
#tf.keras.utils.plot_model(
#models['Parallel_ANN'],
#to_file=r"C:\Users\user\Desktop\iman\Master\RoboticsAndControlSystems\Project\Code\IrrigationControlSystem\model.png",
#show_shapes=True,
#show_dtype=False,
#show_layer_names=True,
#rankdir="TB",
#expand_nested=True,
#dpi=96,
#layer_range=None,
#show_layer_activations=True,
#)

In [ ]:
 
#tf.keras.utils.model_to_dot(
#    models['Cascaded_Out_ANN'],
#    show_shapes=False,
#    show_dtype=False,
#    show_layer_names=True,
#    rankdir="TB",
#    expand_nested=False,
#    dpi=96,
#    subgraph=False,
#    layer_range=None,
#    show_layer_activations=False,
#)